# <span style="color:#a4d4a3">**Grid-based SLAM**</span>

<span style="color:#00703c">***Questions:***</span>

- *Can we solve SLAM problem if <span style="color:#ffa500">**no pre-defined landmarks**</span> are available?*
- *<span style="color:#ffa500">**Are the landmarks useful**</span> to autonomous robots other than for localization?*
- *Can we use the ideas of FastSLAM to <span style="color:#ffa500">**build occupancy maps**</span>?*

<span style="color:#00703c">**Reminder!**</span>

- **Factorization of the SLAM posterior** (Rao-Blackwellization):

$$
p(x_{0:t}, m \mid z_{1:t}, u_{1:t}) = p(x_{0:t} \mid z_{1:t}, u_{1:t}) \cdot p(m \mid x_{1:t}, z_{1:t})
$$

where:

- $p(x_{0:t} \mid z_{1:t}, u_{1:t}) \:\:$ is the path posterior
- $p(m \mid x_{1:t}, z_{1:t}) \:\:$ is the map posterior
 


---

### 🧐 <span style="color:#a4d4a3">**Grid-based Mapping with Known Poses**</span>

- As with landmarks, the <span style="color:#ffa500">**map depends on the poses**</span> of the robot during data acquisition.
- If the poses are <span style="color:#ffa500">**known**</span>, grid-based mapping is <span style="color:#ffa500">**easy**</span> (*"mapping with known poses"*).

Let's see how well that works with a `pygame` example.

- If we put the motion noise to <span style="color:#ffa500">**0.0**</span>, we see what it should look like. 

- If we <span style="color:#ffa500">**increase it**</span>, we see that the odometry poses can not be trusted to build a consistent map.

In [ ]:
import numpy as np
import pygame
from math import cos, sin, pi

# --- Configuration ---
CELL_SIZE = 4
MAX_RANGE = 200
BEAM_STRIDE = 2              # stride along each ray (pixels)
NUM_BEAMS = 180
ANGLE_SUBSAMPLE = 2          # use every k-th beam
NOISE_STD = [1.0, np.deg2rad(0.5)]  # [range_std, bearing_std]
P0, P_FREE, P_HIT = 0.5, 0.3, 0.7
R_WINDOW_CELLS = 2
LO_MAX = 4.0
TRANS_STEP = 12
ROT_STEP = np.deg2rad(12)
motion_noise = [0.05, 0.1, 0.05]

# --- Utility Functions ---
def wrap_angle(a): return (a + np.pi) % (2*np.pi) - np.pi
def logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1 - p))
def logistic(l): return 1.0 / (1.0 + np.exp(-l))

# Vectorized beam casting for a whole scan
def cast_scan_vectorized(origin_xy, angles_world, wall_mask, max_range, ray_stride, range_noise=0.0):
    """
    Build ranges for all beams at once.
    - Out-of-bounds is treated as a hit at boundary (no noise).
    - Range noise is added only when the FIRST hit is a wall (not OOB).
    Returns:
        ranges (B,), first_is_wall (B,)  # whether first hit was an actual wall
    """
    B = angles_world.size
    S = int(max_range // ray_stride) + 1
    r = (np.arange(S, dtype=np.float32) * float(ray_stride))[None, :]  # (1,S)

    ang = angles_world[:, None].astype(np.float32)  # (B,1)
    X = origin_xy[0] + r * np.cos(ang)             # (B,S)
    Y = origin_xy[1] + r * np.sin(ang)             # (B,S)
    X = X.astype(np.int32); Y = Y.astype(np.int32)

    W, H = wall_mask.shape[0], wall_mask.shape[1]
    oob = (X < 0) | (X >= W) | (Y < 0) | (Y >= H)              # (B,S)
    inb = ~oob

    wall = np.zeros_like(oob, dtype=bool)
    if np.any(inb):
        wall[inb] = wall_mask[X[inb], Y[inb]]

    hit = oob | wall
    has_hit = np.any(hit, axis=1)
    first_idx = np.argmax(hit, axis=1)                         # first True along each row
    ranges = np.where(has_hit, r.squeeze()[first_idx], float(max_range)).astype(np.float32)

    # identify type of first hit (wall vs OOB) to add noise only for wall
    first_is_wall = np.zeros(B, dtype=bool)
    mask = has_hit
    if np.any(mask):
        first_is_wall[mask] = wall[np.arange(B)[mask], first_idx[mask]]
        # add noise to wall hits only
        n = np.random.normal(0.0, range_noise, size=int(np.sum(first_is_wall)))
        ranges[first_is_wall] += n.astype(np.float32)

    return ranges, first_is_wall

# --- Motion Model ---
def motion_model_odometry(u, x_prev, noise_std):
    delta_rot1, delta_trans, delta_rot2 = u
    delta_rot1  += np.random.normal(0, noise_std[0])
    delta_trans += np.random.normal(0, noise_std[1])
    delta_rot2  += np.random.normal(0, noise_std[2])
    x = x_prev[0] + delta_trans * np.cos(x_prev[2] + delta_rot1)
    y = x_prev[1] + delta_trans * np.sin(x_prev[2] + delta_rot1)
    theta = wrap_angle(x_prev[2] + delta_rot1 + delta_rot2)
    return np.array([x, y, theta], dtype=np.float32)

# --- Inverse Sensor Model (vectorized deltas) ---
def dL_along_ray(r, z, hit_exists, R_HALF, P0, P_FREE, P_HIT):
    """
    Vectorized log-odds increments along a single ray sample set r (in pixels).
    Returns dL array aligned with r.
    """
    if hit_exists:
        free_mask = (r < z - R_HALF)
        occ_mask  = (r <= z + R_HALF) & ~free_mask
    else:
        free_mask = (r < z)
        occ_mask  = np.zeros_like(free_mask)

    dL = np.zeros_like(r, dtype=np.float32)
    if np.any(free_mask):
        p = P_FREE; dL[free_mask] = logit(p) - logit(P0)
    if np.any(occ_mask):
        p = P_HIT;  dL[occ_mask]  = logit(p) - logit(P0)
    return dL

def update_grid_beam_vectorized(robot_xy, global_angle, z_meas, hit_exists,
                                logodds, w, h, GW, GH, CELL_SIZE, R_HALF, LO_MAX,
                                stride_px=BEAM_STRIDE):
    """
    Vectorized per-beam grid update (no inner loop over cells).
    Deduplicates consecutive grid cells and uses np.add.at to accumulate.
    """
    r_end = int(min(z_meas + (R_HALF if hit_exists else 0.0), MAX_RANGE))
    if r_end <= 0: return

    r = np.arange(0, r_end, max(1, int(stride_px)), dtype=np.float32)  # samples along the ray
    xs = (robot_xy[0] + r * np.cos(global_angle)).astype(np.int32)
    ys = (robot_xy[1] + r * np.sin(global_angle)).astype(np.int32)

    # mask inside image bounds
    valid = (xs >= 0) & (xs < w) & (ys >= 0) & (ys < h)
    if not np.any(valid): return
    xs, ys, r = xs[valid], ys[valid], r[valid]

    # convert to grid cells
    gxs = (xs // CELL_SIZE).astype(np.int32)
    gys = (ys // CELL_SIZE).astype(np.int32)
    gv = (gxs >= 0) & (gxs < GW) & (gys >= 0) & (gys < GH)
    if not np.any(gv): return
    gxs, gys, r = gxs[gv], gys[gv], r[gv]

    # deduplicate consecutive repeated cells
    keep = np.ones(gxs.size, dtype=bool)
    keep[1:] = (gxs[1:] != gxs[:-1]) | (gys[1:] != gys[:-1])
    gxs, gys, r = gxs[keep], gys[keep], r[keep]

    # inverse sensor model deltas (vector)
    dL = dL_along_ray(r, float(z_meas), bool(hit_exists), R_HALF, P0, P_FREE, P_HIT)
    if not np.any(dL): return

    # accumulate and clamp
    np.add.at(logodds, (gxs, gys), dL)
    logodds[gxs, gys] = np.clip(logodds[gxs, gys], -LO_MAX, LO_MAX)

# --- Grid Drawing ---
def render_logodds_surface(logodds):
    p = logistic(logodds)
    gray = (255.0 * (1.0 - p)).astype(np.uint8)  # shape (GW, GH) treated as (W,H)
    gray_rgb = np.dstack([gray, gray, gray])     # shape (GW, GH, 3)
    surf = pygame.surfarray.make_surface(gray_rgb)  # expects (W, H, 3)
    return pygame.transform.scale(surf, (GW * CELL_SIZE, GH * CELL_SIZE))

# --- Initialization ---
pygame.init()
FLOOR_PLAN_PATH = '../figures/floor_plan.png'
floor_plan_orig = pygame.image.load(FLOOR_PLAN_PATH)
w_orig, h_orig = floor_plan_orig.get_size()
scale = min(500 / w_orig, 400 / h_orig)
w, h = int(w_orig * scale), int(h_orig * scale)
screen = pygame.display.set_mode((w * 2, h))
floor_plan = pygame.transform.smoothscale(floor_plan_orig, (w, h))
arr = pygame.surfarray.array3d(floor_plan)             # (W,H,3)
wall_mask = np.all(arr < 128, axis=2)                  # (W,H) True for walls (dark)

GW = (w + CELL_SIZE - 1) // CELL_SIZE
GH = (h + CELL_SIZE - 1) // CELL_SIZE
R_WINDOW = R_WINDOW_CELLS * CELL_SIZE
R_HALF = R_WINDOW / 2.0
L0 = logit(P0)
logodds = np.full((GW, GH), L0, dtype=np.float32)

gt_pose = np.array([w // 2, h // 2, 0.0], dtype=np.float32)
pred_pose = gt_pose.copy()
gt_traj, pred_traj = [gt_pose.copy()], [pred_pose.copy()]
SCAN_EVENT = pygame.USEREVENT + 1
pygame.time.set_timer(SCAN_EVENT, 100)
angles = np.linspace(0, 2 * np.pi, NUM_BEAMS, endpoint=False).astype(np.float32)

clock = pygame.time.Clock()

# --- Main loop ---
running = True
while running:
    dt = clock.tick(30) / 1000.0
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_UP:
                cmd = [0.0, TRANS_STEP, 0.0]
            elif ev.key == pygame.K_LEFT:
                cmd = [-ROT_STEP, 0.0, 0.0]
            elif ev.key == pygame.K_RIGHT:
                cmd = [ ROT_STEP, 0.0, 0.0]
            else:
                cmd = None
            if cmd:
                gt_pose  = motion_model_odometry(cmd, gt_pose, [0,0,0])
                pred_pose= motion_model_odometry(cmd, pred_pose, motion_noise)
                gt_traj.append(gt_pose.copy())
                pred_traj.append(pred_pose.copy())

        elif ev.type == SCAN_EVENT:
            # Subsample beams (vectorized)
            beams_idx = np.arange(0, NUM_BEAMS, ANGLE_SUBSAMPLE)
            a_world   = angles[beams_idx]                              # world-frame angles (B,)

            # Vectorized scan from GT pose (range noise only on wall hits)
            ranges, first_is_wall = cast_scan_vectorized(
                gt_pose[:2], a_world, wall_mask, MAX_RANGE, BEAM_STRIDE, range_noise=NOISE_STD[0]
            )

            # Bearing noise per beam; measurement bearing in robot frame
            bearing_noise = np.random.normal(0.0, NOISE_STD[1], size=a_world.size).astype(np.float32)
            phi = wrap_angle(a_world - pred_pose[2] + bearing_noise)   # (B,)
            # Global measurement direction for map update
            global_angles = pred_pose[2] + phi                         # (B,)

            # Vectorized per-beam grid updates (still a loop over beams; inner ray is vectorized)
            for ang, dist, hit in zip(global_angles, ranges, first_is_wall):
                update_grid_beam_vectorized(pred_pose[:2], ang, dist, bool(hit),
                                            logodds, w, h, GW, GH, CELL_SIZE, R_HALF, LO_MAX,
                                            stride_px=BEAM_STRIDE)

    # --- Drawing (left: floor & GT; right: grid & prediction) ---
    screen.blit(floor_plan, (0, 0))
    if len(gt_traj) >= 2:
        for i in range(1, len(gt_traj)):
            pygame.draw.line(screen, (0, 150, 255),
                             gt_traj[i-1][:2], gt_traj[i][:2], 2)
    gx, gy, gth = gt_pose
    tri = [(gx + 15 * cos(gth),     gy + 15 * sin(gth)),
           (gx + 10 * cos(gth+2.5), gy + 10 * sin(gth+2.5)),
           (gx + 10 * cos(gth-2.5), gy + 10 * sin(gth-2.5))]
    pygame.draw.polygon(screen, (0, 150, 255), tri)

    screen.fill((255, 255, 255), (w, 0, w, h))
    grid_surface = render_logodds_surface(logodds)
    screen.blit(grid_surface, (w, 0))

    px, py = pred_pose[:2]
    pygame.draw.circle(screen, (200, 200, 60), (int(w + px), int(py)), 5)
    if len(pred_traj) >= 2:
        for i in range(1, len(pred_traj)):
            pygame.draw.line(screen, (200, 200, 60),
                             (w + pred_traj[i-1][0], pred_traj[i-1][1]),
                             (w + pred_traj[i][0],   pred_traj[i][1]), 2)
    pygame.display.flip()

pygame.quit()



If we use the poses from only the <span style="color:#ffa500">**raw odometry estimates**</span>, then the result will not be that great. We need <span style="color:#ffa500">**good pose estimates**</span>. Let's see if we can do <span style="color:#ffa500">**grid-based mapping**</span> directly with the Rao-Blackwellized Particle Filter (RBPF). 

---

### ⚡ <span style="color:#a4d4a3">**Grid-based Mapping with RBPFs**</span>

- Each particle represents a <span style="color:#ffa500">**possible trajectory**</span> of the robot.
- Each particle <span style="color:#ffa500">**maintains its own map**</span>.
- Each particle <span style="color:#ffa500">**updates**</span> it upon <span style="color:#ffa500">***"mapping with known poses"***</span>.

<span style="color:#00703c">**Problem**</span>:

- In order to have good poses from the Particle Filter to do the mapping with <span style="color:#ffa500">***"known poses"***</span> we need a <span style="color:#ffa500">**large number of particles**</span> to sufficiently model the motion noise.
- <span style="color:#ffa500">**Increasing the number of samples**</span> is difficult as each map <span style="color:#ffa500">**grows quite large**</span>.

<span style="color:#00703c">**Idea**</span>:

- Improve the pose estimate <span style="color:#ffa500">**before**</span> applying the particle filter.


---


### 🤝 <span style="color:#a4d4a3">**Pose Correction Using Scan-Matching**</span>

A first idea is to maximize the likelihood of the <span style="color:#ffa500">**current pose**</span> and map <span style="color:#ffa500">**relative to the previous pose**</span> and map:

$$
x_t^\star = \arg\max_{x_t} \{ \; p(z_t \mid x_t, m_{t-1}) \; p(x_t \mid u_{t-1}, x_{t-1}^\star) \; \}
$$

where:

- $z_t \:\:$ is the current measurement
- $u_{t-1}\:\:$ is the robot motion
- $m_{t-1}\:\:$ the map constructed so far

> 📝 <span style="color:#0098ff">**Note:**</span> <em>Often the map is a submap consisted of the last 10-20 scnas. </em>

Let's see an example with the following `pygame` where the goal is to reduce odometry drift <span style="color:#ffa500">**before**</span> updating the occupancy grid by aligning the <span style="color:#ffa500">**current scan**</span> to the <span style="color:#ffa500">**current map**</span>.

<span style="color:#00703c">**Inputs**</span>

- Predicted (odometry) pose: $\hat{x}_t = (x, y, \theta)$
- Occupancy grid (log-odds) $L \in \mathbb{R}^{G_W \times G_H}$ and probabilities $p = \sigma(L)$
- Scan $z_t = \{(\phi_i, r_i)\}_{i=1}^N$ in the <span style="color:#ffa500">**robot frame**</span> (bearing, range)

##### <span style="color:#a4d4a3">**Scoring** (MAP objective near odometry)</span>

For a candidate pose $x = (x, y, \theta)$ around $\hat{x}_t$:

1. <span style="color:#00703c">**Transform beam endpoints to world:**</span>
   $$
   p_i(x) =
   \begin{bmatrix} x \\ y \end{bmatrix} +
   R(\theta)
   \begin{bmatrix} r_i \cos\phi_i \\ r_i \sin\phi_i \end{bmatrix},\quad
   R(\theta) =
   \begin{bmatrix}
   \cos\theta & -\sin\theta \\
   \sin\theta & \cos\theta
   \end{bmatrix}.
   $$

2. <span style="color:#00703c">**Correlate with the current map:**</span>
   
   <span style="color:#ffa500">**Discretize**</span> to grid cells $(g_x, g_y) = \lfloor \frac{p_i}{C} \rfloor$ and <span style="color:#ffa500">**accumulate**</span>, where $C$ is the cell size.
   $$
   s_{\text{occ}}(x) = \sum_{i \in \mathcal{V}} p\big( g_x(p_i(x)),\, g_y(p_i(x)) \big),
   $$

   where $\mathcal{V}$ keeps only <span style="color:#ffa500">**in-bounds endpoints**</span>. This correlation can be seen as an <span style="color:#ffa500">**agreement score**</span> between the scan and the map, if displaced by the candidate pose $x$.

3. <span style="color:#00703c">**Add a Gaussian motion prior:**</span>

   $$
   s(x) = s_{\text{occ}}(x)
   - \tfrac{1}{2}\frac{\Delta x^2 + \Delta y^2}{\sigma_{\text{trans}}^2}
   - \tfrac{1}{2}\frac{\Delta\theta^2}{\sigma_{\text{rot}}^2},\quad
   \Delta(\cdot) = x - \hat{x}_t .
   $$

<span style="color:#00703c">**Pose estimate:**</span> pick $x_t^\star = \arg\max_x s(x)$ in a <span style="color:#ffa500">**small neighborhood**</span> of $\hat{x}_t$.

##### <span style="color:#a4d4a3">**Search strategy** (discrete correlative)</span>

- <span style="color:#ffa500">**Coarse → fine**</span> window over $(\Delta x, \Delta y, \Delta\theta)$ around $\hat{x}_t$.
- <span style="color:#ffa500">**Beam subsampling**</span> *(e.g., every 2nd–4th beam)* for speed.
- Optionally <span style="color:#ffa500">**ignore max-range beams**</span> in the score *(limited alignment information)*.

##### <span style="color:#a4d4a3">**Pose update and mapping**</span>

- <span style="color:#ffa500">**Blend**</span> prediction with odometry estimate or <span style="color:#ffa500">**snap**</span> it: $\: x_t \leftarrow \alpha\, x_t^\star + (1-\alpha)\, \hat{x}_t,\quad \alpha \in [0,1].$
- <span style="color:#ffa500">**Mapping with (better) known pose:**</span> integrate the <span style="color:#ffa500">**same**</span> scan into the grid via the inverse sensor model using $x_t$.


In [ ]:
import numpy as np
import pygame
from math import cos, sin, pi

# --- Configuration ---
CELL_SIZE = 4
MAX_RANGE = 250
BEAM_STRIDE = 2          # simulation cast stride [px] along a ray
NUM_BEAMS = 180
NOISE_STD = [1.0, np.deg2rad(0.5)]   # [range_px, bearing_rad]
P0, P_FREE, P_HIT = 0.5, 0.3, 0.7
R_WINDOW_CELLS = 2
LO_MAX = 4.0
TRANS_STEP = 12
ROT_STEP = np.deg2rad(12)
motion_noise = [0.05, 0.1, 0.05]     # [σ_rot1, σ_trans, σ_rot2] (odometry sampling)

# --- Scan-matching tuning ---
SM_ENABLED = True
SM_WARMUP_SCANS = 5          # wait a few scans before matching (map becomes informative)
SM_BEAM_SUBSAMPLE = 2        # use every k-th beam for matching
SM_PRIOR_SIGMA_TRANS = 3.0   # [px] motion prior std for Δx,Δy
SM_PRIOR_SIGMA_ROT   = np.deg2rad(3.0)  # [rad] prior std for Δθ
SM_BLEND = 1.0               # 1.0 => take full correction, 0.5 => blend with odom

# coarse-to-fine search windows (Δx, Δy in px; Δθ in rad)
SM_SEARCH_COARSE = dict(dx=8, dy=8, dth=np.deg2rad(8), step=4, th_step=np.deg2rad(4))
SM_SEARCH_FINE   = dict(dx=3, dy=3, dth=np.deg2rad(3), step=1, th_step=np.deg2rad(1))

# --- Utility Functions ---
def wrap_angle(a):
    return (a + np.pi) % (2*np.pi) - np.pi

def logit(p, eps=1e-6):
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def logistic(l):
    return 1.0/(1.0 + np.exp(-l))

def clamp(v, lo, hi):
    return lo if v < lo else (hi if v > hi else v)

# --- Motion Model ---
def motion_model_odometry(u, x_prev, noise_std):
    dr1, dt, dr2 = u
    dr1 += np.random.normal(0, noise_std[0])
    dt  += np.random.normal(0, noise_std[1])
    dr2 += np.random.normal(0, noise_std[2])
    th  = x_prev[2] + dr1
    x = x_prev[0] + dt * np.cos(th)
    y = x_prev[1] + dt * np.sin(th)
    theta = wrap_angle(th + dr2)
    return np.array([x, y, theta], dtype=np.float32)

# --- Vectorized beam casting (environment oracle on floor plan) ---
def cast_scan_vectorized(origin_xy, angles_world, wall_mask, max_range, ray_stride, range_noise=0.0):
    """
    Build ranges for all beams at once.
    - OOB is treated as a hit at boundary (no noise).
    - Range noise is added only when the FIRST hit is a wall (not OOB).
    Returns:
        ranges (B,), first_is_wall (B,)
    """
    B = angles_world.size
    S = int(max_range // ray_stride) + 1
    r = (np.arange(S, dtype=np.float32) * float(ray_stride))[None, :]  # (1,S)

    ang = angles_world[:, None].astype(np.float32)  # (B,1)
    X = origin_xy[0] + r * np.cos(ang)             # (B,S)
    Y = origin_xy[1] + r * np.sin(ang)             # (B,S)
    X = X.astype(np.int32); Y = Y.astype(np.int32)

    W, H = wall_mask.shape[0], wall_mask.shape[1]
    oob = (X < 0) | (X >= W) | (Y < 0) | (Y >= H)              # (B,S)
    inb = ~oob

    wall = np.zeros_like(oob, dtype=bool)
    if np.any(inb):
        wall[inb] = wall_mask[X[inb], Y[inb]]

    hit = oob | wall
    has_hit = np.any(hit, axis=1)
    first_idx = np.argmax(hit, axis=1)                         # first True per beam
    ranges = np.where(has_hit, r.squeeze()[first_idx], float(max_range)).astype(np.float32)

    # Noise only for walls (not OOB)
    first_is_wall = np.zeros(B, dtype=bool)
    if np.any(has_hit):
        first_is_wall[has_hit] = wall[np.arange(B)[has_hit], first_idx[has_hit]]
        if np.any(first_is_wall):
            ranges[first_is_wall] += np.random.normal(0.0, range_noise, size=int(np.sum(first_is_wall))).astype(np.float32)

    return ranges, first_is_wall

# --- Inverse Sensor Model (vectorized deltas) ---
def dL_along_ray(r, z, hit_exists, R_HALF, P0, P_FREE, P_HIT):
    """
    Vectorized log-odds increments along a single ray sample set r (in pixels).
    Returns dL array aligned with r.
    """
    if hit_exists:
        free_mask = (r < z - R_HALF)
        occ_mask  = (r <= z + R_HALF) & ~free_mask
    else:
        free_mask = (r < z)
        occ_mask  = np.zeros_like(free_mask)

    dL = np.zeros_like(r, dtype=np.float32)
    if np.any(free_mask):
        p = P_FREE; dL[free_mask] = logit(p) - logit(P0)
    if np.any(occ_mask):
        p = P_HIT;  dL[occ_mask]  = logit(p) - logit(P0)
    return dL

# --- Grid Update (vectorized casting to cells; duplicates collapsed) ---
def update_grid_beam_vectorized(robot_xy, global_angle, z_meas, hit_exists, logodds, w, h,
                                GW, GH, CELL_SIZE, R_HALF, LO_MAX, stride_px=BEAM_STRIDE):
    r_end = int(min(z_meas + (R_HALF if hit_exists else 0.0), MAX_RANGE))
    if r_end <= 0: 
        return

    r = np.arange(0, r_end, max(1, int(stride_px)), dtype=np.float32)
    xs = (robot_xy[0] + r * np.cos(global_angle)).astype(np.int32)
    ys = (robot_xy[1] + r * np.sin(global_angle)).astype(np.int32)

    valid = (xs >= 0) & (xs < w) & (ys >= 0) & (ys < h)
    if not np.any(valid): 
        return
    xs, ys, r = xs[valid], ys[valid], r[valid]

    gxs = (xs // CELL_SIZE).astype(np.int32)
    gys = (ys // CELL_SIZE).astype(np.int32)
    gv = (gxs >= 0) & (gxs < GW) & (gys >= 0) & (gys < GH)
    if not np.any(gv): 
        return
    gxs, gys, r = gxs[gv], gys[gv], r[gv]

    # Deduplicate consecutive cells on the ray
    keep = np.ones(gxs.size, dtype=bool)
    keep[1:] = (gxs[1:] != gxs[:-1]) | (gys[1:] != gys[:-1])
    gxs, gys, r = gxs[keep], gys[keep], r[keep]

    # Vectorized inverse sensor model deltas
    dL = dL_along_ray(r, float(z_meas), bool(hit_exists), R_HALF, P0, P_FREE, P_HIT)
    if not np.any(dL): 
        return

    np.add.at(logodds, (gxs, gys), dL)
    logodds[gxs, gys] = np.clip(logodds[gxs, gys], -LO_MAX, LO_MAX)

# --- Likelihood surface from log-odds (optionally smoothed) ---
def occupancy_surface(logodds):
    # probability image in [0,1]
    p = logistic(logodds)
    # light spatial smoothing (3x3 box, twice) to act like a crude likelihood field
    def blur(img):
        pad = np.pad(img, 1, mode='edge')
        return (pad[:-2,:-2]+pad[:-2,1:-1]+pad[:-2,2:]+
                pad[1:-1,:-2]+pad[1:-1,1:-1]+pad[1:-1,2:]+
                pad[2:,  :-2]+pad[2:,  1:-1]+pad[2:,  2:]) / 9.0
    p = blur(blur(p))
    return p  # shape (GW, GH)

# --- Vectorized correlative scan matching on occupancy surface ---
def search_around_vec(pose, beams_xy_robot, occ_surf, CELL_SIZE, GW, GH, win, sig_xy, sig_th):
    """
    Vectorized correlative search over (Δx,Δy,Δθ).
    Returns (best_pose, best_score).
    """
    x0, y0, th0 = float(pose[0]), float(pose[1]), float(pose[2])

    # Candidate grids
    dxs  = np.arange(-win['dx'],  win['dx'] + 1,  win['step'],   dtype=np.float32)  # (M,)
    dys  = np.arange(-win['dy'],  win['dy'] + 1,  win['step'],   dtype=np.float32)  # (N,)
    dths = np.arange(-win['dth'], win['dth'] + win['th_step']/2, win['th_step'], dtype=np.float32)  # (T,)
    if dxs.size == 0 or dys.size == 0 or dths.size == 0:
        return np.array([x0, y0, th0], dtype=np.float32), -1e9

    DX, DY = np.meshgrid(dxs, dys, indexing='ij')                 # (M,N)

    THs = wrap_angle(th0 + dths)                                  # (T,)
    c, s = np.cos(THs).astype(np.float32), np.sin(THs).astype(np.float32)

    # Rotate all beams for all T (broadcast): (T,B)
    bx = beams_xy_robot[:,0].astype(np.float32)[None, :]          # (1,B)
    by = beams_xy_robot[:,1].astype(np.float32)[None, :]          # (1,B)
    RX = (c[:,None]*bx - s[:,None]*by)                            # (T,B)
    RY = (s[:,None]*bx + c[:,None]*by)                            # (T,B)

    # Translate all candidates: world coords for each (T,M,N,B)
    X = x0 + DX[None, :, :, None] + RX[:, None, None, :]
    Y = y0 + DY[None, :, :, None] + RY[:, None, None, :]

    # Convert to grid indices
    gx = np.floor(X / float(CELL_SIZE)).astype(np.int32)
    gy = np.floor(Y / float(CELL_SIZE)).astype(np.int32)

    # Valid mask (inside grid)
    valid = (gx >= 0) & (gx < GW) & (gy >= 0) & (gy < GH)

    # Lookup occupancy probabilities for valid points
    occ_vals = np.zeros_like(X, dtype=np.float32)
    if np.any(valid):
        gcx = gx.clip(0, GW-1); gcy = gy.clip(0, GH-1)
        occ_vals[valid] = occ_surf[gcx[valid], gcy[valid]]

    # Sum over beams → correlation score per candidate (T,M,N)
    scores = np.sum(occ_vals, axis=-1, dtype=np.float32)

    # Add Gaussian motion priors (broadcast)
    prior_xy = -0.5 * ((DX/float(sig_xy))**2 + (DY/float(sig_xy))**2)         # (M,N)
    prior_th = -0.5 * ((dths/float(sig_th))**2).astype(np.float32)            # (T,)
    scores += prior_xy[None, :, :] + prior_th[:, None, None]

    # Argmax and reconstruct best pose
    idx_flat = np.argmax(scores)
    t, m, n = np.unravel_index(idx_flat, scores.shape)
    best_pose = np.array([x0 + dxs[m], y0 + dys[n], THs[t]], dtype=np.float32)
    best_s = float(scores[t, m, n])
    return best_pose, best_s

def scan_match(pred_pose, scan_phis, scan_ranges, occ_surf, CELL_SIZE, GW, GH):
    """
    Vectorized two-stage coarse-to-fine search on occ_surf.
    """
    if scan_phis.size == 0:
        return pred_pose.copy()

    # Precompute beam endpoints in robot frame
    bx = scan_ranges * np.cos(scan_phis)
    by = scan_ranges * np.sin(scan_phis)
    beams_xy_robot = np.stack([bx, by], axis=1).astype(np.float32)  # (B,2)

    p1, _ = search_around_vec(pred_pose, beams_xy_robot, occ_surf, CELL_SIZE, GW, GH,
                              SM_SEARCH_COARSE, sig_xy=SM_PRIOR_SIGMA_TRANS, sig_th=SM_PRIOR_SIGMA_ROT)
    p2, _ = search_around_vec(p1,        beams_xy_robot, occ_surf, CELL_SIZE, GW, GH,
                              SM_SEARCH_FINE,   sig_xy=SM_PRIOR_SIGMA_TRANS, sig_th=SM_PRIOR_SIGMA_ROT)
    return p2

# --- Rendering ---
def render_logodds_surface(logodds, GW, GH, CELL_SIZE):
    # logodds is indexed [gx, gy] = [x_cell, y_cell] with shape (GW, GH)
    p = logistic(logodds)                           # (GW, GH)
    gray = (255.0 * (1.0 - p)).astype(np.uint8)    # (GW, GH)
    rgb  = np.dstack([gray, gray, gray])           # (GW, GH, 3)
    surf = pygame.surfarray.make_surface(rgb)      # expects (W, H, 3)
    return pygame.transform.scale(surf, (GW * CELL_SIZE, GH * CELL_SIZE))

# --- Initialization ---
pygame.init()
FLOOR_PLAN_PATH = '../figures/floor_plan.png'
floor_plan_orig = pygame.image.load(FLOOR_PLAN_PATH)
w_orig, h_orig = floor_plan_orig.get_size()
scale = min(500 / w_orig, 400 / h_orig)
w, h = int(w_orig * scale), int(h_orig * scale)
screen = pygame.display.set_mode((w * 2, h))
floor_plan = pygame.transform.smoothscale(floor_plan_orig, (w, h))
arr = pygame.surfarray.array3d(floor_plan)
wall_mask = np.all(arr < 128, axis=2)   # shape (W,H)

GW = (w + CELL_SIZE - 1) // CELL_SIZE
GH = (h + CELL_SIZE - 1) // CELL_SIZE
R_WINDOW = R_WINDOW_CELLS * CELL_SIZE
R_HALF = R_WINDOW / 2.0
L0 = logit(P0)
logodds = np.full((GW, GH), L0, dtype=np.float32)

gt_pose  = np.array([w // 2, h // 2, 0.0], dtype=np.float32)
pred_pose = gt_pose.copy()
gt_traj, pred_traj = [gt_pose.copy()], [pred_pose.copy()]

angles = np.linspace(0, 2 * np.pi, NUM_BEAMS, endpoint=False).astype(np.float32)
SCAN_EVENT = pygame.USEREVENT + 1
pygame.time.set_timer(SCAN_EVENT, 100)

clock = pygame.time.Clock()
running = True
scan_counter = 0

while running:
    dt = clock.tick(30) / 1000.0

    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_m:
                SM_ENABLED = not SM_ENABLED
            cmd = None
            if ev.key == pygame.K_UP:
                cmd = [0.0, TRANS_STEP, 0.0]
            elif ev.key == pygame.K_LEFT:
                cmd = [-ROT_STEP, 0.0, 0.0]
            elif ev.key == pygame.K_RIGHT:
                cmd = [ ROT_STEP, 0.0, 0.0]
            if cmd:
                gt_pose   = motion_model_odometry(cmd, gt_pose,  [0,0,0])         # noise-free GT
                pred_pose = motion_model_odometry(cmd, pred_pose, motion_noise)   # odometry
                gt_traj.append(gt_pose.copy())
                pred_traj.append(pred_pose.copy())

        elif ev.type == SCAN_EVENT:
            # --- Vectorized full 360° scan from GT pose (environment oracle) ---
            ranges, first_is_wall = cast_scan_vectorized(
                gt_pose[:2], angles, wall_mask, MAX_RANGE, BEAM_STRIDE, range_noise=NOISE_STD[0]
            )

            # Robot-frame bearings with per-beam noise (vectorized)
            bearing_noise = np.random.normal(0.0, NOISE_STD[1], size=angles.size).astype(np.float32)
            phis = wrap_angle(angles - gt_pose[2] + bearing_noise).astype(np.float32)

            # --- Scan matching: refine pred_pose using current map ---
            scan_counter += 1
            if SM_ENABLED and scan_counter >= SM_WARMUP_SCANS:
                occ = occupancy_surface(logodds)
                use = np.arange(0, phis.size, SM_BEAM_SUBSAMPLE)
                ref_pose = scan_match(pred_pose, phis[use], ranges[use], occ, CELL_SIZE, GW, GH)
                pred_pose = (SM_BLEND * ref_pose + (1.0 - SM_BLEND) * pred_pose).astype(np.float32)

            # --- Mapping with (corrected) pose: integrate the same scan ---
            # Use half the beams for map integration for speed, and only mark hits if **wall** (not OOB)
            for i in range(0, phis.size, 2):
                phi, dist = phis[i], ranges[i]
                hit_exists = bool(first_is_wall[i])  # more accurate than (dist < MAX_RANGE)
                global_angle = pred_pose[2] + phi
                update_grid_beam_vectorized(pred_pose[:2], global_angle, dist, hit_exists, logodds,
                                            w, h, GW, GH, CELL_SIZE, R_HALF, LO_MAX,
                                            stride_px=BEAM_STRIDE)

    # --- Draw ---
    screen.blit(floor_plan, (0, 0))
    if len(gt_traj) >= 2:
        for i in range(1, len(gt_traj)):
            pygame.draw.line(screen, (0, 150, 255), gt_traj[i-1][:2], gt_traj[i][:2], 2)
    gx, gy, gth = gt_pose
    tri = [(gx + 15 * cos(gth),     gy + 15 * sin(gth)),
           (gx + 10 * cos(gth + 2.5), gy + 10 * sin(gth + 2.5)),
           (gx + 10 * cos(gth - 2.5), gy + 10 * sin(gth - 2.5))]
    pygame.draw.polygon(screen, (0, 150, 255), tri)

    screen.fill((255, 255, 255), (w, 0, w, h))
    grid_surface = render_logodds_surface(logodds, GW, GH, CELL_SIZE)
    screen.blit(grid_surface, (w, 0))

    px, py = pred_pose[:2]
    pygame.draw.circle(screen, (200, 200, 60), (int(w + px), int(py)), 5)
    if len(pred_traj) >= 2:
        for i in range(1, len(pred_traj)):
            pygame.draw.line(screen, (200, 200, 60),
                             (w + pred_traj[i-1][0], pred_traj[i-1][1]),
                             (w + pred_traj[i][0],   pred_traj[i][1]), 2)

    # HUD
    font = pygame.font.SysFont(None, 20)
    txt = f"ScanMatch [{'ON' if SM_ENABLED else 'OFF'}]  Scans:{scan_counter}  Blend:{SM_BLEND:.2f}"
    screen.blit(font.render(txt, True, (0,0,0)), (w+8, 8))

    pygame.display.flip()

pygame.quit()


---

### 🚀 <span style="color:#a4d4a3">**Grid-based FastSLAM with Improved Odometry**</span>

So the first idea was to use scan matching to <span style="color:#ffa500">**pre-correct the odometry estimates**</span>, and that is what we saw in the example above.

Then as a second idea, we can maybe try to incorporate this pre-correction to FastSLAM 1.0. 

If we try to incorporate this to FastSLAM directly, a <span style="color:#ffa500">**slight problem**</span> arises from a mathematical point of view. If we look at the Particle Filter for FastSLAM, we used the motion model for odometry estimation and then to update the weight we used the observation model for correction. 

So now if we use the  <span style="color:#ffa500">**observation in the proposal distribution**</span>, the weight computations change and will  <span style="color:#ffa500">**no longer hold**</span> as previously derived.

> 📝 <span style="color:#0098ff">**Note:**</span> <em>While it might not be mathematically correct, in practice it does yield a better result than having raw odometry estimations. </em>

A  <span style="color:#ffa500">**better idea**</span> was to perform scan to scan matching in  <span style="color:#ffa500">**smaller sequences**</span> and then use the Particle Filter for estimates  <span style="color:#ffa500">**between the sequences**</span>, to properly stitch them.

-  <span style="color:#ffa500">**Pre-correct**</span> short odometry sequences using scan matching and use them as  <span style="color:#ffa500">**input to FastSLAM**</span>.
- Scan matching provides  <span style="color:#ffa500">**locally consistent**</span> pose correction.
-  <span style="color:#ffa500">**Fewer particles**</span> are needed since the error in the input is smaller.

> 📝 <span style="color:#0098ff">**Note:**</span> <em>Proposed by Hähnel et al. in 2003. </em>

While the previous solution works, it can be seen as an  <span style="color:#ffa500">**ad-hoc solution**</span> to an improved proposal distribution.  

<span style="color:#ffa500">***But can't we do better than that?***</span> 

Now, we are moving towards  <span style="color:#ffa500">**FastSLAM 2.0**</span> that has an improved proposal distribution, but we are going to derive it for the  <span style="color:#ffa500">**grid-based approach**</span>.

 <span style="color:#00703c">***What's Next?***</span>

- Compute an improved proposal that considers <span style="color:#ffa500">**the most recent observation**</span>:

$$
x_t^{[k]} \sim p(x_{1:t-1}^{[k]}, u_{1:t}, z_{1:t})
$$

<span style="color:#ffa500">**Goal:**</span>

- More <span style="color:#00703c">**precise**</span> sampling
- More <span style="color:#00703c">**accurate**</span> maps
- <span style="color:#00703c">**Less**</span> particles needed

---

### 💪 <span style="color:#a4d4a3">**The Optimal Proposal Distribution**</span>

If we know the <span style="color:#ffa500">**map**</span> $m^{[i]}$ so far, from particle $i$, and we know the <span style="color:#ffa500">**current controls**</span> and <span style="color:#ffa500">**current observation**</span>, we can estimate where we are going to end up, as follows:

$$
p(x_t \mid x_{t-1}^{[i]}, m^{[i]}, z_t, u_t) = \frac{p(z_t \mid x_t, m^{[i]}) \cdot p(x_t \mid x_{t-1}^{[i]}, u_t)}{p(z_t \mid x_{t-1}^{[i]}, m^{[i]}, u_t)} \quad\quad (1)
$$

After applying <span style="color:#ffa500">**Bayes' Rule**</span> to the above, we can see familiar terms. On top we have the observation model and the motion estimate. 

In order to build an <span style="color:#ffa500">**efficient algorithm**</span>, we take a deeper look in to the expected distributions of these terms. 

- It is common that the observation model provides a much <span style="color:#ffa500">**more tight local estimate**</span> compared to the motion model that suffers from accumulated drift (wheel slippage, tire pressure etc.). Thus we make a key assumption:

    <span style="color:#00703c">***For laser rangefinders, $\:p(z_t \mid x_t, m^{[i]}) \:$ is typically peaked and dominates the product of Eq. (1).***</span>

Continuing, we can analyze the denominator to the following integral:

$$
p(z_t \mid x_{t-1}^{[i]}, m^{[i]}, u_t) = \int{p(z_t \mid x_t, m^{[i]}) \cdot p(x_t \mid x_{t-1}^{[i]}, u_t) \cdot dx_t} \quad\quad (2)
$$

Now we notice that the <span style="color:#ffa500">**inner part**</span> of the integral is identical to the nominator in Eq. (1). We rewrite it simply as:

$$
p(z_t \mid x_t, m^{[i]}) \cdot p(x_t \mid x_{t-1}^{[i]}, u_t) = \frac{\tau(x)}{\int{\tau(x_t) \cdot dx_t}}
$$

Thus, we have <span style="color:#ffa500">**the same product**</span> of the two terms where one dominates the other. 

- $p(z_t \mid x_t, m^{[i]}) \:$ (measurement) <span style="color:#00703c">**locally**</span> limits the area over which to integrate.
- $p(x_t \mid x_{t-1}^{[i]}, u_t) \:$ (odometry) <span style="color:#00703c">**globally**</span> limits the area over which to integrate.

We can approximate this term by integrating <span style="color:#ffa500">**only in the restricted area**</span> that is defined by the <span style="color:#ffa500">**dominating term**</span>. So our <span style="color:#ffa500">**Proposal Distribution**</span> looks like this:

$$
p(x_t \mid x_{t-1}^{[i]}, m^{[i]}, z_t, u_t) \approx \frac{\tau(x)}{\int_{\{x_t \mid \tau(x_t) > \epsilon \}}{\tau(x_t) \cdot dx_t}} \quad\quad (3)
$$

A few <span style="color:#00703c">**issues**</span> arise:
- For one, we can not evaluate it in a <span style="color:#ffa500">**closed form**</span> to sample from it.
- Secondly, we need it to be <span style="color:#ffa500">**Gaussian**</span> to <span style="color:#ffa500">**efficiently sample**</span> from it and to compute the weights for the target distribution.

Therefore we try to <span style="color:#ffa500">**approximate**</span> Eq. (3) by a Gaussian. 

$$
p(x_t \mid x_{t-1}^{[i]}, m^{[i]}, z_t, u_t) \approx \mathcal{N}(\mu^{[i]}, \Sigma^{[i]})
$$

The steps are as follows:

- Get the <span style="color:#ffa500">**maximum**</span> reported from the <span style="color:#ffa500">**scan matching**</span>.
- Sample <span style="color:#ffa500">**locally around**</span> that.
- <span style="color:#ffa500">**Fit**</span> a Gaussian.

The mean and covariance for each particle are:

$$
\begin{aligned}
\mu^{[i]} &= \frac{1}{\eta} \Sigma_{j=1}^{K} x_j \tau(x_j) \\
\Sigma^{[i]} &= \frac{1}{\eta} \Sigma_{j=1}^{K} (x_j - \mu^{[i]})(x_j - \mu^{[i]})^T \tau(x_j)
\end{aligned}
$$

where $x_j$ are a set of points <span style="color:#ffa500">**sampled around**</span> the point $x^*$ that the scan matching <span style="color:#ffa500">**converged**</span> to.

This Gaussian can now be used for the proposal distribution in the Particle Filter implementation.

##### <span style="color:#a4d4a3">**Computing the Importance Weight**</span>

The same way that we derived the weight in the previous module, and by applying the same as in Eq. (2), we have:

$$
\begin{aligned}
w_t^{[i]} &= w_{t-1}^{[i]} p(z_t \mid x_{t-1}^{[i]}, m^{[i]}, u_t) \\
&= w_{t-1}^{[i]} \int{p(z_t \mid x_t, m^{[i]}) \cdot p(x_t \mid x_{t-1}^{[i]}, u_t) \cdot dx_t} \\
&\approx w_{t-1}^{[i]} \int_{\{x_t \mid \tau(x_t) > \epsilon \}}{\tau(x_t) \cdot dx_t} \\
&\approx w_{t-1}^{[i]} \sum_{j=1}^K \tau(x_j)
\end{aligned}
$$

where $x_j$ are the sampled points <span style="color:#ffa500">**around the maximum**</span> of the likelihood function found by scan-matching.

---

### 🔁 <span style="color:#a4d4a3">**Resampling**</span> 

Resampling at each step <span style="color:#ffa500">**limits the** ***"memory"***</span> of our filter. If we lose 25% of the particles at each time step, this may lead to <span style="color:#ffa500">**losing history of the poses** and therefore diversity in the particles.

<span style="color:#00703c">**Goal:**</span>
- Reduce the resampling actions.

#### <span style="color:#a4d4a3">**Selective Resampling**</span>

Resampling is necessary to <span style="color:#ffa500">**achieve convergence**</span> but it is also dangerous since <span style="color:#ffa500">**important samples might get lost**</span> *("particle depletion")*. Resampling is only useful if particle <span style="color:#ffa500">**weights differ significantly**</span>.

<span style="color:#00703c">**Key question:**<span style="color:#00703c">
- <span style="color:#ffa500">**When**</span> to resample?

The <span style="color:#ffa500">**number of effective particles**</span> is an <span style="color:#ffa500">**empirical measure**</span> of how well the target distribution is approximated by samples drawn from the proposal:

$$
\eta_{eff} = \frac{1}{\sum_i (w_t^{[i]})^2}
$$

- $\eta_{eff}$ describes the <span style="color:#ffa500">**inverse variance**</span> of the normalized particle weights.
- For equal weights, the sample approximation is close to the target.
- If our approximation is close to the target, no resampling is needed.
- We only resample when $\eta_{eff}$ drops <span style="color:#ffa500">**below a given threshold**</span> ($N/2$).

> 📝 <span style="color:#0098ff">**Note:**</span> <em> It is important for the weights to be normalized when computing $\eta_{eff}$.</em>

---

### 🎞️ <span style="color:#a4d4a3">**Grid-based FastSLAM — Quick Recap**</span>

- A Rao–Blackwellized particle filter where each particle carries a <span style="color:#ffa500">**robot pose**</span> and its own <span style="color:#ffa500">**occupancy grid map**</span> (stored in <span style="color:#ffa500">**log-odds**</span>). There are <span style="color:#ffa500">**no per-landmark EKFs**</span> because the map is not a set of features; it’s a grid of Bernoulli cells.

- Laser measurements produce a <span style="color:#ffa500">**sharp likelihood**</span> around the true pose. We exploit this by <span style="color:#ffa500">**scan-matching**</span> each particle’s scan to its <span style="color:#ffa500">**own map**</span> to obtain a high-quality pose hypothesis.

- <span style="color:#00703c">**Core loop** (per particle):</span>
  1) <span style="color:#ffa500">**Predict**</span> pose from odometry (motion model).  
  2) <span style="color:#ffa500">**Scan-match**</span> the current scan to the particle’s grid; build a <span style="color:#ffa500">**Gaussian proposal**</span> around the matched pose.  
  3) <span style="color:#ffa500">**Sample**</span> the new pose from that proposal and <span style="color:#ffa500">**update the particle’s weight**</span> using the measurement likelihood.  
  4) <span style="color:#ffa500">**Update the grid**</span> with an <span style="color:#ffa500">**inverse sensor model**</span>: cells along each beam become <span style="color:#ffa500">**free**</span>, the endpoint (if any) becomes <span style="color:#ffa500">**occupied**</span> (via log-odds increments).  
  5) <span style="color:#ffa500">**Selective resampling**</span> using effective sample size to avoid particle depletion.

- In landmark-based FastSLAM, each feature has a tiny EKF. Here, each <span style="color:#ffa500">**grid cell**</span> is a simple <span style="color:#ffa500">**Bernoulli**</span> variable updated in <span style="color:#ffa500">**log-odds**</span>; <span style="color:#ffa500">**scan-matching**</span> provides the informative update in <span style="color:#ffa500">**pose space**</span>.

- A <span style="color:#ffa500">**better proposal**</span> reduces the number of particles needed, <span style="color:#ffa500">**mitigates drift**</span>, and improves map consistency in laser-friendly environments.

- Use a <span style="color:#ffa500">**likelihood-field**</span> or <span style="color:#ffa500">**correlation**</span> map for fast scan scores; <span style="color:#ffa500">**subsample beams**</span>; <span style="color:#ffa500">**clamp**</span> log-odds to avoid saturation; treat <span style="color:#ffa500">**max-range**</span> returns as free-space only; perform the <span style="color:#ffa500">**endpoint occupied update outside**</span> the inner free-space ray loop.

- Memory/compute scale with <span style="color:#ffa500">**one map per particle**</span>; weak structure or dynamic scenes can degrade scan-matching; poor initial guesses can cause local-minimum failures.


#### 🧮 <span style="color:#a4d4a3">**Grid-based FastSLAM Algorithm**</span> 


><tt> <span style="color:#4D96FF">def</span> <span style="color:#6BCB77">**Grid_FastSLAM**</span>($\color{#ffa500}\mathcal{X}_{t-1}$, $\color{#ffa500}u_t$, $\color{#ffa500}z_t$):
>
>><span style="color:#FF2DD1">1.</span> <span style="color:#e74c3c">for</span> $k=1$ <span style="color:#ffa500">to</span> $N$ <span style="color:#e74c3c">do</span>: $\quad$ <span style="color:#948979"># Loop over all particles</span>
>>
>>><span style="color:#FF2DD1">2.</span> <span style="color:#e74c3c">Let</span> $\big\langle x_{t-1}^{[k]}, m_{t-1}^{[k]}, w_{t-1}^{[k]} \big\rangle$ <span style="color:#ffa500">be particle</span> $k$ <span style="color:#e74c3c">in</span> $\mathcal{X}_{t-1}$ $\quad$ <span style="color:#948979"># Pose, occupancy grid (log-odds), weight</span>
>>>
>>><span style="color:#FF2DD1">3.</span> $\bar{x}_t^{[k]} \sim p(x_t \mid x_{t-1}^{[k]}, u_t) \quad$ <span style="color:#948979"># Odometry prediction (motion model)</span>
>>>
>>><span style="color:#FF2DD1">4.</span> <span style="color:#948979"># Scan matching for improved proposal</span>
>>>
>>><span style="color:#FF2DD1">5.</span> $x^* \leftarrow$ <span style="color:#6BCB77">**ScanMatch**</span>(<span style="color:#ffa500">start</span> <span style="color:#e74c3c">=</span> $\bar{x}_t^{[k]}$, <span style="color:#ffa500">map</span> <span style="color:#e74c3c">=</span> $m_{t-1}^{[k]}$, <span style="color:#ffa500">scan</span> <span style="color:#e74c3c">=</span> $z_t$) $\quad$ <span style="color:#948979"># Maximize likelihood</span>
>>>
>>><span style="color:#FF2DD1">6.</span> <span style="color:#e74c3c">Generate</span> local samples ${x_j}_{j=1}^K$ <span style="color:#ffa500">around</span> $x^*$ $\quad$ <span style="color:#948979"># Grid/perturbations x, y, theta</span>
>>>
>>><span style="color:#FF2DD1">7.</span> <span style="color:#e74c3c">for</span> each $x_j$: $ \tau_j \gets p(z_t \mid x_j, m_{t-1}^{[k]}) \cdot p(x_j \mid x_{t-1}^{[k]}, u_t)$ $\quad$ <span style="color:#948979"># Likelihood × motion</span>
>>>
>>><span style="color:#FF2DD1">8.</span> $\eta=\sum_{j=1}^K \tau_j \quad$ <span style="color:#948979"># Normalizer for weighted fit</span>
>>>
>>><span style="color:#FF2DD1">9.</span> $\mu^{[k]} = \dfrac{1}{\eta}\sum_{j=1}^{K} x_j \, \tau_j \quad$ <span style="color:#948979"># Proposal mean (pose space)</span>
>>>
>>><span style="color:#FF2DD1">10.</span> $\Sigma^{[k]} = \dfrac{1}{\eta}\sum_{j=1}^{K} (x_j-\mu^{[k]})(x_j-\mu^{[k]})^T \, \tau_j \quad$ <span style="color:#948979"># Proposal covariance</span>
>>>
>>><span style="color:#FF2DD1">11.</span> $x_t^{[k]} \sim \mathcal{N}\big(\mu^{[k]}, \Sigma^{[k]}\big) \quad$ <span style="color:#948979"># Sample pose from improved proposal</span>
>>>
>>><span style="color:#FF2DD1">12.</span> $w^{[k]} \leftarrow w_{t-1}^{[k]}\,\sum_{j=1}^{K}\tau_j \quad$ <span style="color:#948979"># Importance weight (approx. integral)</span>
>>>
>>><span style="color:#FF2DD1">13.</span> <span style="color:#948979"># Occupancy grid update (inverse sensor model, log-odds)</span>
>>>
>>><span style="color:#FF2DD1">14.</span> <span style="color:#e74c3c">for</span> <span style="color:#ffa500">each beam</span> $b \in z_t$ <span style="color:#e74c3c">do</span>:
>>>>
>>>><span style="color:#FF2DD1">15.</span> <span style="color:#e74c3c">for</span> <span style="color:#ffa500">each cell</span> $c$ <span style="color:#ffa500">along the ray</span> $(x_t^{[k]}, b)$ <span style="color:#e74c3c">until</span> hit/range <span style="color:#e74c3c">do</span>:
>>>>
>>>>><span style="color:#FF2DD1">16.</span> $L_c \leftarrow L_c + \lambda_{\text{free}} \quad$ <span style="color:#948979"># Free-space evidence</span>
>>>>>
>>>><span style="color:#FF2DD1">17.</span> <span style="color:#e74c3c">endfor</span> <span style="color:#948979"># cells</span>
>>>>>
>>>><span style="color:#FF2DD1">18.</span> <span style="color:#e74c3c">if</span> <span style="color:#ffa500">valid endpoint cell</span> $c^*$ <span style="color:#e74c3c">then</span>:
>>>>
>>>>><span style="color:#FF2DD1">19.</span> $L_{c^*} \leftarrow L_{c^*} + \lambda_{\text{occ}} \quad$ <span style="color:#948979"># Occupied evidence</span>
>>>>
>>>><span style="color:#FF2DD1">20.</span> <span style="color:#e74c3c">endif</span>
>>>>
>>><span style="color:#FF2DD1">21.</span> <span style="color:#e74c3c">endfor</span> <span style="color:#948979"># beams</span>
>>>
>>><span style="color:#FF2DD1">22.</span> $m_t^{[k]} \leftarrow {L_c}$ <span style="color:#948979"># Keep log-odds</span>
>>>
>><span style="color:#FF2DD1">24.</span> <span style="color:#e74c3c">endfor</span>
>>
>><span style="color:#FF2DD1">25.</span> $\{w^{[k]}\}_{k=1 \ldots N} \quad$ <span style="color:#948979"># Normalize</span>
>>
>><span style="color:#FF2DD1">26.</span> $N_\text{eff} = \dfrac{1}{\sum_k (\hat{w}^{[k]})^2} \quad$ <span style="color:#948979"># Compute number of effective particles</span>
>>
>><span style="color:#FF2DD1">27.</span> <span style="color:#e74c3c">if</span> $N_{\text{eff}} < N_{\text{thresh}}$ <span style="color:#e74c3c">then</span>:
>>
>>><span style="color:#FF2DD1">28.</span> $\mathcal{X}_t = $ <span style="color:#6BCB77">**Resample**</span>($\langle x_t^{[k]}, m_t^{[k]}, w^{[k]} \rangle_{k=1 \ldots N}$)
>>
>><span style="color:#FF2DD1">29.</span> <span style="color:#e74c3c">else</span>:
>>
>>><span style="color:#FF2DD1">30.</span> $\mathcal{X}t = \big\{\langle x_t^{[k]}, m_t^{[k]}, w^{[k]} \rangle \big\}_{k=1 \ldots N} \quad$ <span style="color:#948979"># No resampling</span>
>>
>><span style="color:#FF2DD1">31.</span> <span style="color:#e74c3c">endif</span>
>>
>><span style="color:#FF2DD1">32.</span> <span style="color:#e74c3c">return</span> $\mathcal{X}_t$

---

#### 🎲 `Python Example: Grid-based FastSLAM`

Same as before, we split the notebook into **three cells** so you can run and iterate on each layer independently.

1. **Grid-FastSLAM core**
2. **Map & LiDAR helpers**
3. **Pygame app / UI**

##### ⚙ <span style="color:#a4d4a3">**Grid-FastSLAM Core** (RBPF with scan-matching proposal)</span>

Implements a Rao-Blackwellized particle filter over the <span style="color:#ffa500">**robot pose**</span>. Each particle stores its own <span style="color:#ffa500">**occupancy grid**</span> (log-odds) and a cached $\sigma(L)$ for fast scoring.

- <span style="color:#00703c">**Proposal**</span> — odometry + correlative scan matching (coarse to fine, vectorized).  
  1) Start from the odometry-predicted pose $x^\text{pred}_t$.  
  2) Build robot-frame beam endpoints from the current scan (optionally subsampled).  
  3) <span style="color:#ffa500">**Coarse search**</span> on a small 2-D grid $(\Delta x,\Delta y,\Delta\theta)$.
        - Rotate all beams for every $\Delta\theta$ and translate by every $(\Delta x,\Delta y)$ using broadcasting
        - Index the particle’s cached **$\sigma(L)$** (occupancy probs) at those endpoints
        - Sum to get a correlation score per candidate.  
  4) Add a <span style="color:#ffa500">**Gaussian motion prior**</span> penalty:
        - $-\tfrac12(\Delta x^2+\Delta y^2)/\sigma_{xy}^2 - \tfrac12(\Delta\theta^2)/\sigma_\theta^2$
  5) Take the <span style="color:#ffa500">**argmax**</span> candidate, then repeat with a <span style="color:#ffa500">**finer**</span> window centered at that pose.  
  6) Set the refined pose $x^\text{ref}_t$ and optionally <span style="color:#ffa500">**blend**</span> with the prior: 
        - $x_t \leftarrow \alpha\,x^\text{ref}_t + (1-\alpha)\,x^\text{pred}_t$ (with $\alpha\in[0,1]$).

- <span style="color:#00703c">**Weighting**</span> — correlate beam endpoints against the particle’s $\sigma(L)$ map. 

    - For each particle, <span style="color:#ffa500">**transform beam endpoints**</span> by its pose, sample  $\sigma(L)$ at those cells, and <span style="color:#ffa500">**sum**</span> (simple correlation). Convert to a likelihood:  
        - $ \ell_i \propto \exp{(s_i - \max_j s_j)} $ to stabilize numerics.  

    - Multiply into the particle’s weight and <span style="color:#ffa500">**normalize**</span> across particles.

- <span style="color:#00703c">**Mapping**</span> — vectorized log-odds updates with duplicate-cell collapse + round-robin. 
  For each updated beam:  
  1) March radii $r$ along the measured direction and map to pixels $(x,y)$ then to grid $(g_x,g_y)$. 
  2) <span style="color:#ffa500">**Collapse duplicates**</span> along the ray (keep first index where the cell changes).  
  3) Apply a step-window inverse sensor model to get $\Delta L$ per touched cell: free before the hit, hit window around the endpoint, neutral otherwise.  
  4) Accumulate with logodds and <span style="color:#ffa500">**clip**</span> to $[-L_\text{max}, L_\text{max}]$.  
  
  To save time, update the <span style="color:#ffa500">**best**</span> particle plus a few others in <span style="color:#ffa500">**round-robin**</span> each scan; mark $\sigma(L)$ cache <span style="color:#ffa500">**dirty**</span> and refresh lazily.

- <span style="color:#00703c">**Resampling**</span> — systematic (vectorized) when $N_\text{eff}$ is low; weights normalized every step.  
  Compute $N_\text{eff}=1/\sum_i w_i^2$. If $N_\text{eff}<\tau N$, do <span style="color:#ffa500">**systematic resampling**</span>: draw $u_0\sim\mathcal U(0,\tfrac1N)$, form positions $u_k=u_0+\tfrac{k}{N}$, pick ancestors on the CDF, clone particles (copy maps), and reset weights to $1/N$.

- <span style="color:#00703c">**Knobs**</span> — speed/accuracy trade-offs.**  
  - <span style="color:#ffa500">**Beam subsampling**</span> `sm_beam_subsample`, `map_beam_subsample`: fewer beams → faster, noisier scores/updates.  
  - <span style="color:#ffa500">**Stride**</span> `update_stride_px`: larger stride → fewer touched cells, faster but coarser freespace carving.  
  - <span style="color:#ffa500">**Search windows**</span> `sm_search_coarse/fine`: tighter windows → faster, may miss larger drift; looser windows → more robust, slower.  
  - <span style="color:#ffa500">**Particle count**</span> `Np`: more samples → better global coverage, heavier compute.  
  - <span style="color:#ffa500">**Blend factor**</span> `sm_blend`: $1$ trusts scan match; smaller damps corrections (useful with sparse/noisy maps).

In [1]:
# ==========================================================
# Grid-FastSLAM Pygame Demo (GT left | Grid-FastSLAM right)
# ==========================================================

import numpy as np
import pygame

# --- Small helpers ---
def wrap_angle(a: float) -> float:
    return (a + np.pi) % (2*np.pi) - np.pi

def logit(p: float, eps=1e-6) -> float:
    p = np.clip(p, eps, 1 - eps) 
    return np.log(p/(1 - p))

def clamp(v: float, lo: float, hi: float) -> float:
    return lo if v < lo else (hi if v > hi else v)

# --- Config ---
class Config:
    # Window/panels
    panel_w, panel_h = 500, 400
    window_size = (panel_w*2, panel_h)

    # LiDAR / visualization
    num_beams   = 360
    fov_deg     = 360
    scan_ms     = 120
    max_range_px= 300
    traj_max    = 200

    # Discretization / grid map
    cell_size   = 4           # pixels per cell
    lo_max      = 4.0         # clamp log-odds
    p0          = 0.5
    p_free      = 0.30
    p_hit       = 0.70
    r_window_cells = 2        # +/- cells around hit considered occupied

    # Particle filter
    Np = 50
    resample_neff_ratio = 0.5
    seed = 0

    # Motion (keyboard odometry: [dr1, dt, dr2])
    trans_step = 12.0
    rot_step   = np.deg2rad(12.0)
    odom_noise = np.array([0.05, 0.10, 0.05], dtype=np.float32)

    # --- FAST MODE switches ---
    fast_mode = True
    match_top_k = 1           # match best K particles each scan (1 = only best)
    match_every  = 1          # perform matching every k-th scan (1 = every scan)
    sm_beam_subsample = 4     # use every k-th beam for matching/scoring
    map_beam_subsample = 2    # beams used for map integration
    update_stride_px = 2      # ray stride in pixels when updating grid
    update_particles_per_scan = 3  # best + (this-1) others in round-robin
    sm_search_coarse = dict(dx=6, dy=6, dth=np.deg2rad(6), step=3, th_step=np.deg2rad(3))
    sm_search_fine   = dict(dx=2, dy=2, dth=np.deg2rad(2), step=1, th_step=np.deg2rad(1))

    # Optional blend of correction with prior (pose smoothing)
    sm_blend = 1.0

    # Floor plan path (environment oracle)
    floor_img_path = "../figures/floor_plan.png"

# --- Inverse Sensor Model (vectorized deltas) ---
def p_occ_beam_vec(r: np.ndarray, z: float, hit_exists: bool, R_HALF: float,
                   P0: float, P_FREE: float, P_HIT: float) -> np.ndarray:
    """
    Vectorized log-odds increments along a single ray sample set r (in pixels).
    Args:
        r: (S,) array of ranges along ray [px]
        z: measured range [px]
        hit_exists: whether a hit was measured (True if z < max_range)
        R_HALF: half the thickness of the occupied region [px]
        P0: prior occupancy probability
        P_FREE: free-space probability
        P_HIT: occupied-space probability
    Returns:
        dL: (S,) array of log-odds increments aligned with r
    """
    if hit_exists:
        free_mask = (r < z - R_HALF)
        occ_mask  = (r <= z + R_HALF) & ~free_mask
    else:
        free_mask = (r < z)
        occ_mask  = np.zeros_like(free_mask)
    dL = np.zeros_like(r, dtype=np.float32)
    if np.any(free_mask):
        p = P_FREE; dL[free_mask] = (np.log(p/(1-p)) - np.log(P0/(1-P0)))
    if np.any(occ_mask):
        p = P_HIT;  dL[occ_mask]  = (np.log(p/(1-p)) - np.log(P0/(1-P0)))
    return dL

# --- Grid Update (vectorized casting to cells; duplicates collapsed) ---
def update_grid_beam_vectorized(robot_xy: np.ndarray, global_angle: float,
                                z_meas: float, hit_exists: bool,
                                logodds: np.ndarray, w: int, h: int,
                                GW: int, GH: int, CELL_SIZE: int,
                                R_HALF: float, LO_MAX: float, 
                                P0: float = 0.5, P_FREE: float = 0.3, P_HIT: float = 0.7,
                                stride_px: int = 2):
    """
    Vectorized per-beam grid update (no inner loop over cells).
    Args:
        robot_xy: (2,) robot position in pixels [x,y]
        global_angle: beam angle in world frame [rad]
        z_meas: measured range [px]
        hit_exists: whether a hit was measured (True if z_meas < max_range)
        logodds: (GW, GH) grid log-odds to update in-place
        w, h: floor plan size in pixels
        GW, GH: grid size in cells
        CELL_SIZE: cell size in pixels
        R_HALF: half the thickness of the occupied region [px]
        LO_MAX: clamp log-odds to +/- LO_MAX
        P0: prior occupancy probability
        P_FREE: free-space probability
        P_HIT: occupied-space probability
        stride_px: ray stride in pixels when updating grid
    Returns:
        None (logodds updated in-place)
    """
    r_end = int(min(z_meas + (R_HALF if hit_exists else 0.0), np.hypot(w, h)))
    if r_end <= 0: return
    r = np.arange(0, r_end, max(1, int(stride_px)), dtype=np.float32)   # (S,)

    xs = (robot_xy[0] + r * np.cos(global_angle)).astype(np.int32)
    ys = (robot_xy[1] + r * np.sin(global_angle)).astype(np.int32)
    valid = (xs >= 0) & (xs < w) & (ys >= 0) & (ys < h)
    if not np.any(valid): return
    xs, ys, r = xs[valid], ys[valid], r[valid]

    gxs = xs // CELL_SIZE; gys = ys // CELL_SIZE
    gv = (gxs >= 0) & (gxs < GW) & (gys >= 0) & (gys < GH)
    if not np.any(gv): return
    gxs, gys, r = gxs[gv], gys[gv], r[gv]

    # Deduplicate consecutive cells along the ray
    keep = np.ones(gxs.size, dtype=bool)
    keep[1:] = (gxs[1:] != gxs[:-1]) | (gys[1:] != gys[:-1])
    gxs, gys, r = gxs[keep], gys[keep], r[keep]

    dL = p_occ_beam_vec(r, float(z_meas), bool(hit_exists), R_HALF, P0, P_FREE, P_HIT)
    if not np.any(dL): return

    # Accumulate & clamp
    np.add.at(logodds, (gxs, gys), dL)
    logodds[gxs, gys] = np.clip(logodds[gxs, gys], -LO_MAX, LO_MAX)

# --- Likelihood surface from log-odds (optionally smoothed) ---
def render_logodds_surface(logodds: np.ndarray, GW: int, GH: int, CELL_SIZE: int) -> pygame.Surface:
    """
    Render log-odds grid as a grayscale pygame surface.
    Args:
        logodds: (GW, GH) grid log-odds
        GW, GH: grid size in cells
        CELL_SIZE: cell size in pixels
    Returns:
        surf: pygame.Surface of size (GW*CELL_SIZE, GH*CELL_SIZE)
    """
    # compute only for display (best particle)
    p = 1.0/(1.0 + np.exp(-logodds))
    gray = (255.0 * (1.0 - p)).astype(np.uint8)
    rgb  = np.dstack([gray, gray, gray])
    surf = pygame.surfarray.make_surface(rgb)
    return pygame.transform.scale(surf, (GW*CELL_SIZE, GH*CELL_SIZE))

# --- Vectorized correlative scan matching on occupancy surface ---
def score_pose_from_occ(x: float, y: float, th: float,
                        beams_xy_robot: np.ndarray,
                        occ: np.ndarray, CELL_SIZE: int, GW: int, GH: int) -> float:
    """
    Score a single pose on the occupancy surface by summing occupied prob at beam endpoints.
    Args:
        x, y, th: pose to score
        beams_xy_robot: (B,2) beam endpoints in robot frame
        occ: (GW, GH) occupancy surface (probabilities in [0,1])
        CELL_SIZE: cell size in pixels
        GW, GH: grid size in cells
    Returns:
        score: float score (higher is better); -1e9 if no valid beams
    """
    c, s = np.cos(th), np.sin(th)
    R = np.array([[c,-s],[s,c]], dtype=np.float32)
    pts = (R @ beams_xy_robot.T).T
    pts[:,0] += x; pts[:,1] += y
    gx = np.floor_divide(pts[:,0].astype(np.int32), CELL_SIZE)
    gy = np.floor_divide(pts[:,1].astype(np.int32), CELL_SIZE)
    valid = (gx >= 0) & (gx < GW) & (gy >= 0) & (gy < GH)
    if not np.any(valid): return -1e9
    return float(np.sum(occ[gx[valid], gy[valid]]))

# --- Correlative scan matching (vectorized over all candidates) ---
def search_around(pose: np.ndarray,
                  beams_xy_robot: np.ndarray,
                  occ: np.ndarray, CELL_SIZE: int, GW: int, GH: int,
                  win: dict, sig_xy: float, sig_th: float) -> tuple:
    """
    Vectorized correlative search over (Δx,Δy,Δθ).
    Args:
        pose: (3,) initial pose [x,y,theta]
        beams_xy_robot: (B,2) beam endpoints in robot frame
        occ: (GW, GH) occupancy surface (probabilities in [0,1])
        CELL_SIZE: cell size in pixels
        GW, GH: grid size in cells
        win: dict with search window params: dx, dy, dth, step, th_step
        sig_xy: stddev of Gaussian prior on (x,y) [px]
        sig_th: stddev of Gaussian prior on theta [rad]
    Returns:
        best_pose: (3,) best pose found [x,y,theta]
        best_s: float best score
    """
    x0, y0, th0 = float(pose[0]), float(pose[1]), float(pose[2])

    # Candidate grids
    dxs  = np.arange(-win['dx'],  win['dx'] + 1,  win['step'],    dtype=np.float32)  # (M,)
    dys  = np.arange(-win['dy'],  win['dy'] + 1,  win['step'],    dtype=np.float32)  # (N,)
    dths = np.arange(-win['dth'], win['dth'] + win['th_step']/2,  win['th_step'], dtype=np.float32)  # (T,)
    if dxs.size == 0 or dys.size == 0 or dths.size == 0:
        return np.array([x0, y0, th0], dtype=np.float32), -1e9

    DX, DY = np.meshgrid(dxs, dys, indexing='ij')                  # (M,N)

    THs  = (th0 + dths + np.pi) % (2*np.pi) - np.pi               # (T,)
    c    = np.cos(THs).astype(np.float32)                          # (T,)
    s    = np.sin(THs).astype(np.float32)                          # (T,)

    # Rotate all beams for all T (broadcast): (T,B)
    bx = beams_xy_robot[:,0].astype(np.float32)[None, :]           # (1,B)
    by = beams_xy_robot[:,1].astype(np.float32)[None, :]           # (1,B)
    RX = (c[:,None]*bx - s[:,None]*by)                             # (T,B)
    RY = (s[:,None]*bx + c[:,None]*by)                             # (T,B)

    # Translate all candidates: (T,M,N,B)
    X = x0 + DX[None, :, :, None] + RX[:, None, None, :]
    Y = y0 + DY[None, :, :, None] + RY[:, None, None, :]

    # Grid indices
    gx = np.floor(X / float(CELL_SIZE)).astype(np.int32)
    gy = np.floor(Y / float(CELL_SIZE)).astype(np.int32)

    # Valid mask (inside grid)
    valid = (gx >= 0) & (gx < GW) & (gy >= 0) & (gy < GH)

    # Lookup occupancy probabilities at all indices (occ is σ(L))
    occ_vals = np.zeros_like(X, dtype=np.float32)
    if np.any(valid):
        gcx = gx.clip(0, GW-1); gcy = gy.clip(0, GH-1)
        occ_vals[valid] = occ[gcx[valid], gcy[valid]]

    # Sum over beams → correlation score per candidate (T,M,N)
    scores = np.sum(occ_vals, axis=-1, dtype=np.float32)

    # Add Gaussian priors (broadcast)
    prior_xy  = -0.5 * ((DX/float(sig_xy))**2 + (DY/float(sig_xy))**2)        # (M,N)
    prior_th  = -0.5 * ((dths/float(sig_th))**2).astype(np.float32)           # (T,)
    scores += prior_xy[None, :, :] + prior_th[:, None, None]

    # Argmax over (T,M,N)
    idx_flat = np.argmax(scores)
    t, m, n = np.unravel_index(idx_flat, scores.shape)
    best_pose = np.array([x0 + dxs[m], y0 + dys[n],
                          (THs[t] + np.pi) % (2*np.pi) - np.pi], dtype=np.float32)
    best_s = float(scores[t, m, n])
    return best_pose, best_s

# --- Scan matching (two-stage coarse-to-fine) ---
def scan_match(pred_pose: np.ndarray,
               scan_phis: np.ndarray,
               scan_ranges: np.ndarray,
               occ: np.ndarray, CELL_SIZE: int, GW: int, GH: int,
               cfg: Config) -> tuple:
    """
    Two-stage correlative scan matching on occupancy surface.
    Args:
        pred_pose: (3,) initial pose [x,y,theta]
        scan_phis: (B,) beam angles in robot frame [rad]
        scan_ranges: (B,) beam ranges [px]
        occ: (GW, GH) occupancy surface (probabilities in [0,1])
        CELL_SIZE: cell size in pixels
        GW, GH: grid size in cells
        cfg: Config object with search params
    Returns:
        best_pose: (3,) best pose found [x,y,theta]
        best_s: float best score
    """
    if scan_phis.size == 0: return pred_pose.copy(), 0.0
    bx = scan_ranges * np.cos(scan_phis); by = scan_ranges * np.sin(scan_phis)
    beams_xy_robot = np.stack([bx, by], axis=1).astype(np.float32)
    p1, _  = search_around(pred_pose, beams_xy_robot, occ, CELL_SIZE, GW, GH,
                           cfg.sm_search_coarse, sig_xy=3.0, sig_th=np.deg2rad(3))
    p2, s2 = search_around(p1,       beams_xy_robot, occ, CELL_SIZE, GW, GH,
                           cfg.sm_search_fine,   sig_xy=3.0, sig_th=np.deg2rad(3))
    return p2, float(s2)

# --- Particle class with cached occupancy surface ---
class Particle:
    __slots__ = ("x", "y", "th", "w", "logodds", "occ_cache", "dirty")
    def __init__(self, x, y, th, w, logodds):
        self.x, self.y, self.th = float(x), float(y), float(th)
        self.w = float(w)
        self.logodds = logodds  # (GW,GH) np.float32
        self.occ_cache = 1.0/(1.0 + np.exp(-self.logodds))  # σ(L)
        self.dirty = False

# --- Cached occupancy surface view (update if dirty) ---
def occ_view(pr: "Particle"):
    """
    Return the cached occupancy surface of the particle, updating if dirty.
    Args:
        pr: Particle instance
    Returns:
        occ_cache: (GW, GH) occupancy surface (probabilities in [0,1
    """
    if pr.dirty:
        pr.occ_cache[...] = 1.0/(1.0 + np.exp(-pr.logodds))
        pr.dirty = False
    return pr.occ_cache

# --- Resampling (systematic, vectorized) ---
def resample_systematic_vec(weights: np.ndarray, rng: np.random.Generator) -> np.ndarray:
    """
    Systematic resampling (vectorized).
    Args:
        weights: (N,) array of particle weights (not necessarily normalized)
        rng: np.random.Generator instance
    Returns:
        idx: (N,) array of resampled indices
    """
    N = weights.size
    c = np.cumsum(weights)
    u0 = rng.uniform(0.0, 1.0/N)
    us = u0 + (np.arange(N, dtype=np.float64)/N)
    idx = np.searchsorted(c, us, side="left").astype(np.int32)
    return idx

# --- Grid FastSLAM class ---
class GridFastSLAM:
    def __init__(self, cfg: Config, map_w, map_h):
        self._display_idx = 0
        self._display_frames = 0
        self.cfg = cfg
        self.rng = np.random.default_rng(cfg.seed)
        self.GW = (map_w + cfg.cell_size - 1) // cfg.cell_size
        self.GH = (map_h + cfg.cell_size - 1) // cfg.cell_size
        self.L0 = logit(cfg.p0)
        self.p = []
        self.scan_counter = 0
        self._rr = 0  # round-robin pointer

    # --- Particle filter steps ---
    def global_initialize(self, gx: float, gy: float, th: float,
                          pos_sd: float, th_sd_deg: float):
        """
        Initialize particles around (gx,gy,th) with Gaussian noise.
        Args:
            gx, gy, th: mean pose [x,y,theta]
            pos_sd: stddev of position noise [px]
            th_sd_deg: stddev of heading noise [deg]
        Returns:
            None
        """
        self.p.clear()
        for _ in range(self.cfg.Np):
            x  = gx + self.rng.normal(0, pos_sd)
            y  = gy + self.rng.normal(0, pos_sd)
            thp= wrap_angle(th + self.rng.normal(0, np.deg2rad(th_sd_deg)))
            logodds = np.full((self.GW, self.GH), self.L0, dtype=np.float32)
            self.p.append(Particle(x, y, thp, 1.0/self.cfg.Np, logodds))

    def predict(self, u):
        dr1, dt, dr2 = u
        n = self.cfg.odom_noise
        for pr in self.p:
            dr1s = dr1 + self.rng.normal(0, n[0])
            dts  = dt  + self.rng.normal(0, n[1])
            dr2s = dr2 + self.rng.normal(0, n[2])
            th   = pr.th + dr1s
            pr.x += dts*np.cos(th); pr.y += dts*np.sin(th)
            pr.th = wrap_angle(th + dr2s)

    def update(self, scan_phis, scan_ranges, map_w, map_h):
        self.scan_counter += 1
        use = np.arange(0, scan_phis.size, max(1, int(self.cfg.sm_beam_subsample)))
        ph = scan_phis[use]; rg = scan_ranges[use]

        # ---- (A) Scan-matching ----
        idx_sorted = np.argsort([-pr.w for pr in self.p])
        to_match = idx_sorted[:max(1, int(self.cfg.match_top_k))]
        do_match_now = (self.scan_counter % max(1, int(self.cfg.match_every))) == 0

        raw_scores = np.zeros(len(self.p), dtype=np.float32)

        if self.cfg.fast_mode and do_match_now:
            for j in to_match:
                pr = self.p[j]
                pose_pred = np.array([pr.x, pr.y, pr.th], dtype=np.float32)
                # match against this particle's cached occ (σ(L))
                ref_pose, s_best = scan_match(pose_pred, ph, rg,
                                              occ_view(pr), self.cfg.cell_size, self.GW, self.GH, self.cfg)
                new_pose = (self.cfg.sm_blend*ref_pose + (1.0-self.cfg.sm_blend)*pose_pred).astype(np.float32)
                pr.x, pr.y, pr.th = float(new_pose[0]), float(new_pose[1]), float(new_pose[2])
                raw_scores[j] = s_best

        # ---- (B) Likelihood / weights (cheap scoring from cached occ) ----
        bx = rg*np.cos(ph); by = rg*np.sin(ph)
        beams_xy_robot = np.stack([bx, by], axis=1).astype(np.float32)

        for i, pr in enumerate(self.p):
            s = score_pose_from_occ(pr.x, pr.y, pr.th, beams_xy_robot,
                                    occ_view(pr), self.cfg.cell_size, self.GW, self.GH)
            raw_scores[i] = max(raw_scores[i], s)

        m = float(np.max(raw_scores))
        like = np.exp(raw_scores - m)
        for pr, lf in zip(self.p, like):
            pr.w *= float(lf + 1e-8)

        # ---- (C) Map updates: round-robin a few particles, always include best ----
        R_HALF = (self.cfg.r_window_cells * self.cfg.cell_size) / 2.0
        upd_idxs = [idx_sorted[0]]  # best always
        need = max(0, int(self.cfg.update_particles_per_scan) - 1)
        for t in range(need):
            j = (self._rr + t) % len(self.p)
            if j not in upd_idxs: upd_idxs.append(j)
        self._rr = (self._rr + need) % len(self.p)

        for j in upd_idxs:
            pr = self.p[j]
            x,y,th = pr.x, pr.y, pr.th
            for i in range(0, scan_phis.size, max(1, int(self.cfg.map_beam_subsample))):
                phi = float(scan_phis[i]); dist = float(scan_ranges[i])
                hit_exists = (dist < self.cfg.max_range_px - 1e-3)
                global_angle = th + phi
                update_grid_beam_vectorized(
                    np.array([x,y], dtype=np.float32), global_angle, dist, hit_exists,
                    pr.logodds, map_w, map_h, self.GW, self.GH, self.cfg.cell_size,
                    R_HALF, self.cfg.lo_max, stride_px=self.cfg.update_stride_px
                )
            pr.dirty = True  # mark occ cache stale

        # ---- (D) Normalize & resample ----
        self.normalize_weights()
        if self.neff() < self.cfg.resample_neff_ratio * len(self.p):
            self.resample_systematic()

    def normalize_weights(self):
        w = np.array([pr.w for pr in self.p], dtype=np.float64)
        s = float(np.sum(w))
        if s <= 0 or not np.isfinite(s):
            u = 1.0/len(self.p)
            for pr in self.p: pr.w = u
        else:
            for pr, wi in zip(self.p, w/s): pr.w = float(wi)

    def neff(self):
        w = np.array([pr.w for pr in self.p], dtype=np.float64)
        return float(1.0/np.sum((w+1e-12)**2))

    # --- Resampling ---
    def resample_systematic(self):
        """
        Systematic resampling (vectorized).
        """
        N = len(self.p)
        w = np.array([pr.w for pr in self.p], dtype=np.float64)
        idxs = resample_systematic_vec(w, self.rng)  # vectorized
        # clone particles
        new = [Particle(self.p[j].x, self.p[j].y, self.p[j].th, 1.0/N, self.p[j].logodds.copy()) for j in idxs]
        # copy caches too (safe but optional; will refresh on first occ_view if dirty)
        for q in new:
            q.occ_cache = 1.0/(1.0 + np.exp(-q.logodds)); q.dirty = False
        self.p = new

    # --- Pose estimation & display ---
    def estimate_pose(self, top_k_frac:int=1.0):
        """
        Estimate pose as weighted average of top-K particles.
        Args:
            top_k_frac: fraction of particles to use (1.0 = all, 0.5 = top half, etc.)
        Returns:
            pose: (3,) estimated pose [x,y,theta]
        """
        N = len(self.p); k = max(1, int(N*top_k_frac))
        idx = np.argsort([-pr.w for pr in self.p])[:k]
        ws = np.array([self.p[i].w for i in idx], dtype=np.float64)
        ws = ws / (np.sum(ws) + 1e-12)
        xs = np.array([self.p[i].x for i in idx]); ys = np.array([self.p[i].y for i in idx])
        ths= np.array([self.p[i].th for i in idx])
        x = float(np.sum(ws*xs)); y = float(np.sum(ws*ys))
        th = float(np.arctan2(np.sum(ws*np.sin(ths)), np.sum(ws*np.cos(ths))))
        return np.array([x,y,th], dtype=np.float32)

    def best_particle(self):
        return self.p[int(np.argmax([pr.w for pr in self.p]))]

    def fused_display_surface(self, cell_size: int) -> pygame.Surface:
        """
        Render the fused occupancy surface from all particles as a grayscale pygame surface.
        Args:
            cell_size: cell size in pixels for display scaling
        Returns:
            surf: pygame.Surface of size (GW*cell_size, GH*cell_size)
        """
        # P(cell=occ) ≈ Σ w_i * σ(L_i)
        P = np.zeros((self.GW, self.GH), dtype=np.float32)
        for pr in self.p:
            P += pr.w * occ_view(pr)
        gray = (255.0 * (1.0 - P)).astype(np.uint8)  # free=white, occ=dark
        rgb  = np.dstack([gray, gray, gray])
        surf = pygame.surfarray.make_surface(rgb)
        return pygame.transform.scale(surf, (self.GW*cell_size, self.GH*cell_size))

    def sticky_best_index(self, margin=1.10, max_hold=10):
        ws = np.array([pr.w for pr in self.p], dtype=np.float64)
        j_best = int(np.argmax(ws))
        if self._display_frames < max_hold:
            if ws[j_best] > ws[self._display_idx] * margin:
                self._display_idx = j_best
                self._display_frames = 0
            else:
                self._display_frames += 1
        else:
            self._display_idx = j_best
            self._display_frames = 0
        return self._display_idx


pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


##### 🗺️ <span style="color:#a4d4a3">**Map & LiDAR Helpers**</span> (environment oracle + vectorized scan builder)

Provides the floor-plan map and a fast, vectorized ray caster to generate scans.

- <span style="color:#ffa500">**Map2D:**</span> loads/rescales an image; builds a boolean <span style="color:#ffa500">**wall mask**</span> (dark = wall). Includes `nearest_free_to`.

- <span style="color:#ffa500">**Scan builder:**</span> `cast_scan_vectorized` computes all ranges at once; returns first-hit or boundary distances.

- <span style="color:#ffa500">**Parameters:**</span> image path, ray stride, number of beams, max range.


In [2]:
# ------------------------------------------------------------
# Cell 2 — Map & LiDAR helpers (Map2D, cast_scan_vectorized)
# ------------------------------------------------------------
import numpy as np
import pygame

# --- 2D Map from floor plan image ---
class Map2D:
    def __init__(self, panel_w:int, panel_h:int, floor_img_path:str, ray_stride:int=2):
        """
        Load floor plan image and create wall mask.
        Args:
            panel_w, panel_h: display panel size in pixels
            floor_img_path: path to floor plan image file
            ray_stride: ray stride in pixels when casting scans
        """
        self.ray_stride = ray_stride
        self.surface_orig = pygame.image.load(floor_img_path)
        w0,h0 = self.surface_orig.get_size()
        scale = min(panel_w / w0, panel_h / h0)
        self.w, self.h = int(w0*scale), int(h0*scale)
        self.surface = pygame.transform.smoothscale(self.surface_orig, (self.w, self.h))
        arr = pygame.surfarray.array3d(self.surface)  # (W,H,3)
        self.wall_mask = np.all(arr < 128, axis=2)    # walls at dark pixels

    # --- Cast a single scan ---
    def nearest_free_to(self, p: tuple) -> tuple:
        """
        Find nearest free cell to (x,y) in pixels (if p is in wall).
        Args:
            p: (x,y) point in pixels
        Returns:   
        (x,y) nearest free point in pixels
        """
        x0,y0 = int(p[0]), int(p[1])
        x0 = np.clip(x0, 0, self.w-1); y0 = np.clip(y0, 0, self.h-1)
        if not self.wall_mask[x0,y0]: return (x0,y0)
        for r in range(1, max(self.w, self.h)):
            for x in np.clip([x0-r, x0, x0+r], 0, self.w-1):
                for y in np.clip([y0-r, y0, y0+r], 0, self.h-1):
                    if not self.wall_mask[x,y]: return (int(x),int(y))
        return (x0,y0)

# --- Vectorized ray casting for full scan ---
def cast_scan_vectorized(origin_xy: np.ndarray,
                         beam_angles_world: np.ndarray,
                         wall_mask: np.ndarray,
                         max_range_px: int,
                         ray_stride: int) -> tuple:
    """
    Build the full scan by casting rays in the wall mask (vectorized).
    Args:
        origin_xy: (2,) origin of rays in pixels [x,y]
        beam_angles_world: (B,) beam angles in world frame [rad]
        wall_mask: (W,H) boolean wall mask (True=wall, False=free)
        max_range_px: maximum range of rays [px]
        ray_stride: ray stride in pixels when casting rays
    Returns:
        ranges: (B,) array of ranges [px] (max_range_px if no hit)
        has_hit: (B,) boolean array indicating whether a hit was found
    """
    B = beam_angles_world.size
    S = int(max_range_px // ray_stride) + 1
    r = (np.arange(S, dtype=np.float32) * float(ray_stride))[None, :]  # (1,S)

    ang = beam_angles_world[:, None]           # (B,1)
    X = origin_xy[0] + r * np.cos(ang)         # (B,S)
    Y = origin_xy[1] + r * np.sin(ang)         # (B,S)
    X = X.astype(np.int32); Y = Y.astype(np.int32)

    W, H = wall_mask.shape[0], wall_mask.shape[1]
    oob = (X < 0) | (X >= W) | (Y < 0) | (Y >= H)    # (B,S)
    inb = ~oob
    wall = np.zeros_like(oob, dtype=bool)
    if np.any(inb):
        wall[inb] = wall_mask[X[inb], Y[inb]]

    hit = oob | wall
    has_hit = np.any(hit, axis=1)
    first_idx = np.argmax(hit, axis=1)  # 0 if none; guard with has_hit
    ranges = np.where(has_hit, r.squeeze()[first_idx], float(max_range_px))
    return ranges.astype(np.float32), has_hit


##### 🎮 <span style="color:#a4d4a3">**Pygame App / UI** (two-panel visualizer)</span>

Wires the map and core into an interactive demo.

- <span style="color:#00703c">**Left panel:**</span> ground-truth robot on the floor-plan (keyboard odometry).
- <span style="color:#00703c">**Right panel:**</span> best particle’s <span style="color:#ffa500">**occupancy grid**</span>, particle cloud, estimated pose, and trajectory.
- <span style="color:#00703c">**Loop:**</span> keyboard → GT motion → RBPF <span style="color:#ffa500">**predict**</span> → vectorized <span style="color:#ffa500">**scan**</span> → RBPF <span style="color:#ffa500">**update**</span> → draw.
- <span style="color:#00703c">**Hotkeys:**</span> <span style="color:#ffa500">**M**</span> toggle fast mode, <span style="color:#ffa500">**P**</span> toggle particles, <span style="color:#ffa500">**G**</span> re-init, <span style="color:#ffa500">**K**</span> reset, <span style="color:#ffa500">**S**</span> save map.


In [5]:
#--- Main application with Pygame ---

import sys, os
import numpy as np
import pygame

# -------------------- small helpers used by UI --------------------
def i2(p): return (int(p[0]), int(p[1]))

# -------------------- panel --------------------
class Panel:
    def __init__(self, screen, rect): self.screen, self.rect = screen, rect
    @property
    def ox(self): return self.rect[0]
    def clear(self, color):
        pygame.draw.rect(self.screen, color, self.rect)
        pygame.draw.rect(self.screen, (200,200,200), self.rect, 2)
    def blit(self, surf, at=(0,0)):
        x,y,_,_ = self.rect; self.screen.blit(surf, (x+at[0], y+at[1]))
    def polygon(self, color, pts):
        x,y,_,_ = self.rect; P = [(x+int(px), y+int(py)) for (px,py) in pts]
        pygame.draw.polygon(self.screen, color, P)
    def circle(self, color, p, r, width=0):
        x,y,_,_ = self.rect
        pygame.draw.circle(self.screen, color, (x+int(p[0]), y+int(p[1])), int(r), width)
    def line(self, color, p0, p1, w=1):
        x,y,_,_ = self.rect; pygame.draw.line(self.screen, color, (x+int(p0[0]), y+int(p0[1])), (x+int(p1[0]), y+int(p1[1])), w)
    def text(self, font, s, color, pos):
        x,y,_,_ = self.rect; self.screen.blit(font.render(s, True, color), (x+pos[0], y+pos[1]))

# -------------------- App --------------------
class App:
    def __init__(self, cfg: Config):
        pygame.init()
        self.cfg = cfg
        self.screen = pygame.display.set_mode(cfg.window_size)
        pygame.display.set_caption("GT (left) | Grid-FastSLAM (right, static map)")
        self.clock = pygame.time.Clock()
        self.font  = pygame.font.SysFont(None, 18)
        self.rng   = np.random.default_rng(cfg.seed)

        # Panels
        self.left  = Panel(self.screen, (0, 0, cfg.panel_w, cfg.panel_h))
        self.right = Panel(self.screen, (cfg.panel_w, 0, cfg.panel_w, cfg.panel_h))

        # Map + LiDAR (oracle)
        self.map = Map2D(cfg.panel_w, cfg.panel_h, cfg.floor_img_path, ray_stride=2)

        # RBPF (grid-based)
        self.fs = GridFastSLAM(cfg, self.map.w, self.map.h)

        # Start at center (nearest free)
        cx, cy = cfg.panel_w//2, cfg.panel_h//2
        gx, gy = self.map.nearest_free_to((cx, cy))
        self.gt_pose = np.array([gx, gy, 0.0], dtype=np.float32)
        self.fs.global_initialize(gx, gy, 0.0, th_sd_deg=10.0, pos_sd=3.0)
        self.est_pose = np.array([gx, gy, 0.0], dtype=np.float32)

        self.gt_traj, self.est_traj = [], []
        self.gt_traj.append(self.gt_pose.copy())
        self.est_traj.append(self.est_pose.copy())

        # Scan timer
        self.SCAN_EVENT = pygame.USEREVENT + 1
        pygame.time.set_timer(self.SCAN_EVENT, cfg.scan_ms)

        # UI toggles
        self.draw_particles = True
        self.sm_enabled = cfg.fast_mode  # tie to fast_mode for clarity

        # Precompute scan angles (world frame); bearings computed per scan
        self.angles_world = np.linspace(0, 2*np.pi, cfg.num_beams, endpoint=False)

    @staticmethod
    def motion_model_odometry(u, x_prev: np.ndarray) -> np.ndarray:
        dr1, dt, dr2 = u
        x_new = x_prev[0] + dt*np.cos(x_prev[2] + dr1)
        y_new = x_prev[1] + dt*np.sin(x_prev[2] + dr1)
        th_new= wrap_angle(x_prev[2] + dr1 + dr2)
        return np.array([x_new, y_new, th_new], dtype=np.float32)

    def step_motion(self, cmd):
        self.gt_pose = self.motion_model_odometry(cmd, self.gt_pose)
        self.gt_traj.append(self.gt_pose.copy()); self.gt_traj = self.gt_traj[-self.cfg.traj_max:]
        self.fs.predict(cmd)
        self.est_pose = self.fs.estimate_pose(top_k_frac=1.0)
        self.est_traj.append(self.est_pose.copy()); self.est_traj = self.est_traj[-self.cfg.traj_max:]

    def step_scan(self):
        # Vectorized scan from GT pose using floor-plan oracle
        ranges, _ = cast_scan_vectorized(self.gt_pose[:2], self.angles_world,
                                         self.map.wall_mask, self.cfg.max_range_px,
                                         ray_stride=self.map.ray_stride)
        phis = wrap_angle(self.angles_world - self.gt_pose[2])  # bearings in robot frame

        # RBPF update
        self.fs.update(phis.astype(np.float32), ranges.astype(np.float32),
                       self.map.w, self.map.h)
        self.est_pose = self.fs.estimate_pose(top_k_frac=1.0)

    # --- Drawing ---
    def draw_left(self):
        self.left.clear((30,30,30))
        self.left.blit(self.map.surface, (0,0))
        if len(self.gt_traj) >= 2:
            for i in range(1, len(self.gt_traj)):
                p0 = i2(self.gt_traj[i-1][:2]); p1 = i2(self.gt_traj[i][:2])
                pygame.draw.line(self.screen, (0,150,255), (self.left.ox+p0[0], p0[1]),
                                 (self.left.ox+p1[0], p1[1]), 2)
        gx,gy,gth = map(float, self.gt_pose)
        pts = [
            (gx + 15*np.cos(gth),     gy + 15*np.sin(gth)),
            (gx + 10*np.cos(gth+2.5), gy + 10*np.sin(gth+2.5)),
            (gx + 10*np.cos(gth-2.5), gy + 10*np.sin(gth-2.5))
        ]
        self.left.polygon((0,150,255), pts)

    def draw_right(self):
        self.right.clear((255,255,255))
        # Sticky best (reduces flicker); or use fused_display_surface(...) instead
        j = self.fs.sticky_best_index(margin=1.10, max_hold=10)
        best = self.fs.p[j]
        grid_surface = render_logodds_surface(best.logodds, self.fs.GW, self.fs.GH, self.cfg.cell_size)
        # grid_surface = self.fs.fused_display_surface(self.cfg.cell_size)
        self.right.blit(grid_surface, (0,0))
        if len(self.est_traj) >= 2:
            for i in range(1, len(self.est_traj)):
                p0 = self.est_traj[i-1][:2]; p1 = self.est_traj[i][:2]
                self.right.line((120,200,160), p0, p1, 2)
        if self.draw_particles:
            for pr in self.fs.p:
                self.right.circle((0,180,120), (pr.x, pr.y), 2)
        px,py,pth = map(float, self.est_pose)
        self.right.circle((0,180,120), (px,py), 6, width=2)
        hx = px + 14*np.cos(pth); hy = py + 14*np.sin(pth)
        self.right.line((0,180,120), (px,py), (hx,hy), 2)

        fps = self.clock.get_fps()
        neff = self.fs.neff()
        hud = (f"FPS:{fps:4.1f} | N:{self.cfg.Np} | Neff:{neff:5.1f} | "
               f"Beams:{self.cfg.num_beams}/SM:{self.cfg.sm_beam_subsample}/MAP:{self.cfg.map_beam_subsample} | "
               f"Cell:{self.cfg.cell_size}px | FastMode:{'ON' if self.cfg.fast_mode else 'OFF'}")
        self.right.text(self.font, hud, (50,255,10), (10,0))

    def run(self):
        running = True
        while running:
            self.clock.tick(30)  # target FPS
            for ev in pygame.event.get():
                if ev.type == pygame.QUIT:
                    running = False
                elif ev.type == pygame.KEYDOWN:
                    cmd = None
                    if   ev.key == pygame.K_UP:    cmd = np.array([0.0, self.cfg.trans_step, 0.0], dtype=np.float32)
                    elif ev.key == pygame.K_LEFT:  cmd = np.array([-self.cfg.rot_step, 0.0, 0.0], dtype=np.float32)
                    elif ev.key == pygame.K_RIGHT: cmd = np.array([ self.cfg.rot_step, 0.0, 0.0], dtype=np.float32)
                    elif ev.key == pygame.K_p:     self.draw_particles = not self.draw_particles
                    elif ev.key == pygame.K_m:     self.cfg.fast_mode = not self.cfg.fast_mode  # toggle fast
                    elif ev.key == pygame.K_s:
                        best = self.fs.best_particle()
                        surf = render_logodds_surface(best.logodds, self.fs.GW, self.fs.GH, self.cfg.cell_size)
                        out = "grid_fastslam_map.png"
                        pygame.image.save(surf, out); print(f"[saved] {out}")
                    elif ev.key == pygame.K_g:
                        cx, cy = self.cfg.panel_w//2, self.cfg.panel_h//2
                        gx, gy = self.map.nearest_free_to((cx, cy))
                        self.fs.global_initialize(gx, gy, 0.0, th_sd_deg=10.0, pos_sd=3.0)
                        self.est_pose = np.array([gx, gy, 0.0], dtype=np.float32)
                        self.est_traj.clear()
                    elif ev.key == pygame.K_k:
                        cx, cy = self.cfg.panel_w//2, self.cfg.panel_h//2
                        gx, gy = self.map.nearest_free_to((cx, cy))
                        self.gt_pose  = np.array([gx, gy, 0.0], dtype=np.float32)
                        self.est_pose = self.gt_pose.copy()
                        self.gt_traj.clear(); self.est_traj.clear()
                    if cmd is not None: self.step_motion(cmd)
                elif ev.type == pygame.USEREVENT + 1:
                    self.step_scan()

            self.draw_left(); self.draw_right(); pygame.display.flip()

        pygame.quit(); sys.exit()

# -------------------- run --------------------
if __name__ == "__main__":
    cfg = Config()
    App(cfg).run()


SystemExit: 

/home/niksta/Documents/Projects/a-brief-slam-summary/slam/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


---

#### ✔️ <span style="color:#a4d4a3">Summary</span>

- Approach to SLAM that combines <span style="color:#ffa500">**scan matching**</span> and <span style="color:#ffa500">**FastSLAM**</span>.

- Scan matching to generate virtual <span style="color:#ffa500">**"high quality"** **motion estimates**</span>.

- Can be seen as an <span style="color:#ffa500">**ad-hoc solution**</span> to an <span style="color:#ffa500">**improved proposal distribution**</span>.

- The idea of FastSLAM can also be applied in the context of grid maps.

- Improved proposals are essential.

- Similar to scan matching on a per particle base

- <span style="color:#ffa500">**Selective resamples**</span> reduce the risk of particle depletion.

- Substantial reduction of the <span style="color:#ffa500">**required number of particles**</span>.

---

### 📚 <span style="color:#a4d4a3">**Reading Material**</span>

**Grid-FastSLAM with Improved Proposals**
- Grisetti et al.: ***"Improved Techniques for Grid Mapping with Rao-Blackwellized Particle Filters"***, *2007*
- Stachniss et al.: ***"Analyzing Gaussian Proposal Distributions for Mapping"***, *2007*

**Grid-FastSLAM & Scan-Matching**
- Hähnel et al.: ***"An Efficient FastSLAM Algorithm for Generating Maps of Large-Scale Cyclic Environments from Raw Laser Range Measurements"***, *2003*
